# Type Inference for Python

## Representing Typing Constraints

We use the following data structures to represent typing constraints based on our premise of inferring types for Python variables by focusing on their interfaces (methods).

First, we use `TypeHandle`'s, opaque pointers to point to *concrete typing constraints* for each encountered variable. This level of indirection allows us to modify the typing constraints for a variable in a low coupling manner, i.e., the typing constraints for other variables interact with those of this variable via this variable's `TypeHandle`, rather than this variable's typing constraints themselves.

A `TypeHandle` can point to any of the following *concrete typing constraints* for a variable:

- A `TypeInferenceUnknown` represents that we know no typing constraints for a variable.
- A `TypeInferenceClass` stores the methods we know about a variable. Each method is represented using the `TypeHandle`'s of the parameters and return values.
- A type annotation extracted from `typeshed`.

In [1]:
from attrs import define, field

from type_annotation import *


class TypeHandle(object):
    pass

TypeInferenceUnknown = type(None)

@define
class TypeInferenceMethod:
    parameter_type_handle_list: list
    return_value_type_handle: TypeHandle

@define
class TypeInferenceClass:
    present_method_name_to_type_inference_method_dict: dict
    absent_method_name_to_type_inference_method_dict: dict

@define
class TypeInferenceGlobalFunctionObject:
    global_function: GlobalFunction

@define
class TypeInferenceConcreteClassObject:
    concrete_class: ConcreteClass

Furthermore, in the following cases, we assume that a set of `TypeHandle`'s should *resolve to the same type*.

- `TypeHandle`'s in an assignment.
- `TypeHandle`'s pointing to elements when constructing containers with the `build_tuple`, `build_list`, and `build_set` expressions, as well as those pointing to keys and values used with the `build_map` expression.
- `TypeHandle`'s in a Phi node.
- Given a particular method, `TypeHandle`'s passed to the `same` parameter or corresponding to the method's return values.

In these cases, we associate these `TypeHandle`'s with an external `TypeVariable` that is *not* extracted from `typeshed`.

## Storing Typing Constraints

We use the following data structures to store the aforementioned typing constraints.

- A `dict` that maps variable names to their `TypeHandle`'s.
- A `dict` that maps `TypeHandle`'s to *concrete typing constraints*.
- A `dict` that maps external `TypeVariable`'s to `TypeHandle`'s that should *resolve to the same type*.

In [2]:
variable_name_to_type_handle_dict = dict()
type_handle_to_concrete_typing_constraints_dict = dict()
type_variable_to_type_handle_set_dict = dict()

## Updating Typing Constraints

We use the following functions to effectively update typing constraints when we visit expressions in Numba IR according to our typing rules.

- *get type handle for variable*, which retrieves the `TypeHandle` for a variable or creates it if it does not exist. This function is called for each variable encountered in a Numba IR expression.
- *set to type annotation*, which tries to set the *concrete typing constraints* pointed to by a `TypeHandle` to a type annotation. If the original *concrete typing constraints* form a *structural supertype* of the type annotation, they are updated. Otherwise, they are left unchanged.
- *add present method*, called whenever we infer the presence of a method within a variable according to our typing rules.

Additionally, should a `TypeHandle` point to a `TypeVariable` that signals several other `TypeHandle`'s should *resolve to the same type*, calls to *set to type annotation* and *add present method* are broadcasted on those `TypeHandle`'s as well.


In [3]:
import sys

from look_up import *
from subtyping import *


def get_type_handle_for_variable(name: str, indent_level=0):
    global variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict
    
    indent = '    ' * indent_level
    print(indent, f'get_type_handle_for_variable {name}', file=sys.stderr)
    
    if name in variable_name_to_type_handle_dict:
        return variable_name_to_type_handle_dict[name]
    else:
        type_handle = TypeHandle()
        
        variable_name_to_type_handle_dict[name] = type_handle
        type_handle_to_concrete_typing_constraints_dict[type_handle] = TypeInferenceUnknown()
        
        return type_handle


# Used to stop infinite recursion
SET_TO_TYPE_ANNOTATION_PENDING_TYPE_HANDLES_SET = set()

def set_to_type_annotation(type_handle: TypeHandle, type_annotation, indent_level=0):
    global variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict, SET_TO_TYPE_ANNOTATION_PENDING_TYPE_HANDLES_SET

    if indent_level > 100:
        assert False, 'infinite recursion suspected'
    
    indent = '    ' * indent_level
    print(indent, f'set_to_type_annotation {type_handle} {type_annotation}', file=sys.stderr)

    assert isinstance(type_annotation, (TypeVariable, ConcreteClass, Subscription, TypeInferenceGlobalFunctionObject, TypeInferenceConcreteClassObject)), f'unsupported type_annotation: {type_annotation}'
    
    if isinstance(type_annotation, (TypeVariable, Subscription)):
        for type_variable in iterate_type_variables_in_type_annotation(type_annotation, indent_level + 1):
            if type_variable not in type_variable_to_type_handle_set_dict:
                assert False, f'{type_variable} in {type_annotation} not in type_variable_to_type_handle_set_dict'
                
    if type_handle in SET_TO_TYPE_ANNOTATION_PENDING_TYPE_HANDLES_SET:
        print(indent, f'{type_handle} in SET_TO_TYPE_ANNOTATION_PENDING_TYPE_HANDLES_SET, skipped', file=sys.stderr)
        return
    else:
        SET_TO_TYPE_ANNOTATION_PENDING_TYPE_HANDLES_SET.add(type_handle)
    
    concrete_typing_constraints = type_handle_to_concrete_typing_constraints_dict[type_handle]
    
    if isinstance(concrete_typing_constraints, TypeInferenceUnknown):
        if isinstance(type_annotation, TypeVariable):
            # Associate the type_handle with the `TypeVariable`
            print(indent, f'associating {type_handle} with {type_annotation}', file=sys.stderr)
            new_type_handle = TypeHandle()
            type_handle_to_concrete_typing_constraints_dict[new_type_handle] = concrete_typing_constraints
            type_variable_to_type_handle_set_dict[type_annotation].add(new_type_handle)
            
            type_handle_to_concrete_typing_constraints_dict[type_handle] = type_annotation
        else:
            print(indent, f'{concrete_typing_constraints} -> {type_annotation}', file=sys.stderr)
            type_handle_to_concrete_typing_constraints_dict[type_handle] = type_annotation
    elif isinstance(concrete_typing_constraints, ConcreteClass):
        if isinstance(type_annotation, TypeVariable):
            # Associate the type_handle with the `TypeVariable`
            print(indent, f'associating {type_handle} with {type_annotation}', file=sys.stderr)
            new_type_handle = TypeHandle()
            type_handle_to_concrete_typing_constraints_dict[new_type_handle] = concrete_typing_constraints
            type_variable_to_type_handle_set_dict[type_annotation].add(new_type_handle)
            
            type_handle_to_concrete_typing_constraints_dict[type_handle] = type_annotation
        elif isinstance(type_annotation, ConcreteClass):
            if type_annotation_subtyping(concrete_typing_constraints, type_annotation, indent_level + 1)[0]:
                print(indent, f'{concrete_typing_constraints} is subtype of {type_annotation}, no action taken', file=sys.stderr)
            else:
                print(indent, f'{concrete_typing_constraints} -> {type_annotation}', file=sys.stderr)
                type_handle_to_concrete_typing_constraints_dict[type_handle] = type_annotation
        else:
            assert False, f'unsupported type_annotation to assign to {concrete_typing_constraints}: {type_annotation}'
    # Recursively handle `TypeHandle`'s associated with the `TypeVariable` before handling the `TypeVariable` itself
    elif isinstance(concrete_typing_constraints, TypeVariable):
        print(indent, f'{concrete_typing_constraints} -> {type_annotation}', file=sys.stderr)
        
        type_handle_set = type_variable_to_type_handle_set_dict[concrete_typing_constraints]
        for type_handle_ in type_handle_set:
            assert type_handle != type_handle_
            set_to_type_annotation(type_handle_, type_annotation, indent_level + 1)
        
        for type_handle_ in type_handle_to_concrete_typing_constraints_dict:
            concrete_typing_constraints_ = type_handle_to_concrete_typing_constraints_dict[type_handle_]
            if isinstance(concrete_typing_constraints_, (TypeVariable, ConcreteClass, Subscription)):
                new_concrete_typing_constraints_ = replace_type_variables_in_type_annotation(concrete_typing_constraints_, {
                    concrete_typing_constraints: type_annotation
                })
                type_handle_to_concrete_typing_constraints_dict[type_handle_] = new_concrete_typing_constraints_
        
        type_variable_to_type_handle_set_dict.pop(concrete_typing_constraints, None)
    elif isinstance(concrete_typing_constraints, TypeInferenceClass):
        if isinstance(type_annotation, TypeVariable):
            # Associate the type_handle with the `TypeVariable`
            print(indent, f'associating {type_handle} with {type_annotation}', file=sys.stderr)
            new_type_handle = TypeHandle()
            type_handle_to_concrete_typing_constraints_dict[new_type_handle] = concrete_typing_constraints
            type_variable_to_type_handle_set_dict[type_annotation].add(new_type_handle)
            
            type_handle_to_concrete_typing_constraints_dict[type_handle] = type_annotation
        elif isinstance(type_annotation, ConcreteClass):
            class_definition = look_up_class(type_annotation)
            for method_name, type_inference_method in concrete_typing_constraints.present_method_name_to_type_inference_method_dict.items():
                if method_name not in class_definition.method_name_to_method_list_dict:
                    assert False, f'cannot set {present_method_name_to_type_inference_method_dict} to {type_annotation}: method {method_name} not present in {type_annotation}'
                else:
                    method_definition_list = class_definition.method_name_to_method_list_dict[method_name]
                    method_definition = method_definition_list[0]
                    
                    update_type_handles_in_type_inference_method_according_to_function_definition(
                        type_inference_method,
                        method_definition,
                        indent_level + 1,
                        is_method=True
                    )
            
            print(indent, f'{concrete_typing_constraints} -> {type_annotation}', file=sys.stderr)
            type_handle_to_concrete_typing_constraints_dict[type_handle] = type_annotation
        else:
            assert False, f'unsupported type_annotation to assign to {concrete_typing_constraints}: {type_annotation}'
    else:
        assert False, f'unsupported concrete_typing_constraints pointed to by type_handle: {concrete_typing_constraints}'
        
    SET_TO_TYPE_ANNOTATION_PENDING_TYPE_HANDLES_SET.remove(type_handle)


def add_present_method(type_handle: TypeHandle, method_name: str, method: TypeInferenceMethod, indent_level=0):
    global variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict
    
    indent = '    ' * indent_level
    print(indent, f'add_present_method {type_handle} {method_name} {method}', file=sys.stderr)
    
    concrete_typing_constraints = type_handle_to_concrete_typing_constraints_dict[type_handle]
    print(indent, f'concrete_typing_constraints: {concrete_typing_constraints}', file=sys.stderr)
    
    if isinstance(concrete_typing_constraints, TypeInferenceUnknown):
        # Convert concrete_typing_constraints to TypeInferenceClass with a present method
        type_handle_to_concrete_typing_constraints_dict[type_handle] = TypeInferenceClass(
            present_method_name_to_type_inference_method_dict={ method_name: method },
            absent_method_name_to_type_inference_method_dict=dict()
        )
        return True
    elif isinstance(concrete_typing_constraints, TypeInferenceClass):
        assert method_name not in concrete_typing_constraints.absent_method_name_to_type_inference_method_dict, f'{method_name} previously determined to be absent'

        if method_name not in concrete_typing_constraints.present_method_name_to_type_inference_method_dict:
            concrete_typing_constraints.present_method_name_to_type_inference_method_dict[method_name] = method
        else:
            # if method is already present, we associate TypeHandle's passed to the same parameter or corresponding to the method's return values with TypeVariable's.
            method_ = concrete_typing_constraints.present_method_name_to_type_inference_method_dict[method_name]
            parameter_type_handle_list_ = method_.parameter_type_handle_list
            return_value_type_handle_ = method_.return_value_type_handle

            assert len(parameter_type_handle_list_) == len(method.parameter_type_handle_list)

            new_parameter_type_handle_list = [ parameter_type_handle_list_[0] ]

            for (parameter_type_handle_, parameter_type_handle) in zip(parameter_type_handle_list_[1:], method.parameter_type_handle_list[1:]):
                parameter_type_variable, new_parameter_type_handle = associate_with_type_variable([parameter_type_handle_, parameter_type_handle])
                new_parameter_type_handle_list.append(new_parameter_type_handle)
            
            return_value_type_variable, new_return_value_type_handle = associate_with_type_variable([return_value_type_handle_, method.return_value_type_handle])

            concrete_typing_constraints.present_method_name_to_type_inference_method_dict[method_name] = TypeInferenceMethod(new_parameter_type_handle_list, new_return_value_type_handle)
    elif isinstance(concrete_typing_constraints, (ConcreteClass, Subscription)):
        # A ConcreteClass or Subscription references the instance of a class we already have full information about
        if isinstance(concrete_typing_constraints, ConcreteClass):
            class_definition = look_up_class(concrete_typing_constraints)
        elif isinstance(concrete_typing_constraints, Subscription):
            class_definition = instantiate_type_variables_in_class_definition(
            look_up_class(concrete_typing_constraints.concrete_class),
            concrete_typing_constraints.type_annotation_tuple
            )

        # The added method is already present
        # Thus, we do not modify the information we know about the ConcreteClass or Subscription
        # Instead, we extract information about the method from it
        # And use information about the method to update type information for the parameters and return value
        if method_name in class_definition.method_name_to_method_list_dict:
            method_definition_list = class_definition.method_name_to_method_list_dict[method_name]
            method_definition = method_definition_list[0]
            
            update_type_handles_in_type_inference_method_according_to_function_definition(
                method,
                method_definition,
                indent_level + 1,
                is_method=True
            )
        
        # The added method is not present
        # Convert the ConcreteClass or Subscription back to a TypeInferenceClass and add the method
        else:
            print(indent, 'Convert the ConcreteClass or Subscription back to a TypeInferenceClass and add the method', file=sys.stderr)
            present_method_name_to_type_inference_method_dict = dict()

            for method_name_, method_list_ in class_definition.method_name_to_method_list_dict.items():
                method_definition_ = method_list_[0]

                parameter_type_handle_list_ = [type_handle]
                for parameter_type_annotation_ in method_definition_.parameter_type_annotation_list[1:]:
                    parameter_type_handle_ = TypeHandle()
                    type_handle_to_concrete_typing_constraints_dict[parameter_type_handle_] = parameter_type_annotation_
                    parameter_type_handle_list_.append(parameter_type_handle_)
                
                return_value_type_handle_ = TypeHandle()
                type_handle_to_concrete_typing_constraints_dict[return_value_type_handle_] = method_definition_.return_value_type_annotation

                method_ = TypeInferenceMethod(parameter_type_handle_list=parameter_type_handle_list_, return_value_type_handle=return_value_type_handle_)
                present_method_name_to_type_inference_method_dict[method_name_] = method_
            
            # Add the new method
            present_method_name_to_type_inference_method_dict[method_name] = method

            # Create the TypeInferenceClass
            type_handle_to_concrete_typing_constraints_dict[type_handle] = TypeInferenceClass(
                present_method_name_to_type_inference_method_dict=present_method_name_to_type_inference_method_dict,
                absent_method_name_to_type_inference_method_dict=dict()
            )
    elif isinstance(concrete_typing_constraints, TypeInferenceGlobalFunctionObject):
        # A TypeInferenceGlobalFunctionObject references the instance of a global function we already have full information about
        # Any added method must already be present, and that method must be `__call__`
        # Thus, we do not modify the information we know about the TypeInferenceGlobalFunctionObject
        # Instead, we extract information about the method from it
        # And use information about the method to update type information for the parameters and return value
        assert method_name == '__call__'
        
        global_function = concrete_typing_constraints.global_function
        function_definition_list = look_up_global_function(global_function, indent_level + 1)
        
        function_definition = function_definition_list[0]

        update_type_handles_in_type_inference_method_according_to_function_definition(
            method,
            function_definition,
            indent_level + 1
        )
    elif isinstance(concrete_typing_constraints, TypeInferenceConcreteClassObject):
        # A TypeInferenceConcreteClassObject references the instance of a concrete class we already have full information about
        # Any added method must already be present, and that method must be `__call__`
        # Thus, we do not modify the information we know about the TypeInferenceConcreteClassObject
        # Instead, we extract information about the method from it
        # And use information about the method to update type information for the parameters and return value
        assert method_name == '__call__'
        
        concrete_class = concrete_typing_constraints.concrete_class
        class_definition_ = look_up_class(concrete_class, indent_level + 1)

        # If the class definition contains TypeVariable's
        # A fresh set of TypeVariable's should be initialized
        if class_definition_.type_variable_list:
            new_type_variable_list = list()

            for old_type_variable in class_definition_.type_variable_list:
                new_type_variable = TypeVariable()
                type_variable_to_type_handle_set_dict[new_type_variable] = set()
                new_type_variable_list.append(new_type_variable)
            
            class_definition = instantiate_type_variables_in_class_definition(
                class_definition_,
                new_type_variable_list,
                indent_level + 1
            )
        else:
            class_definition = class_definition_
        
        # Find the constructor
        if '__init__' in class_definition.method_name_to_method_list_dict:
            constructor_method_list = class_definition.method_name_to_method_list_dict['__init__']
        elif '__new__' in class_definition.method_name_to_method_list_dict:
            constructor_method_list = class_definition.method_name_to_method_list_dict['__new__']
        else:
            assert False, f'no constructor found for {concrete_class}'
        
        constructor_definition = constructor_method_list[0]
        
        update_type_handles_in_type_inference_method_according_to_function_definition(
            method,
            constructor_definition,
            indent_level + 1,
            is_method=True
        )
    elif isinstance(concrete_typing_constraints, TypeVariable):
        # Recursively handle `TypeHandle`'s associated with the `TypeVariable`
        assert concrete_typing_constraints in type_variable_to_type_handle_set_dict
        for type_handle_ in type_variable_to_type_handle_set_dict[concrete_typing_constraints]:
            parameter_type_handle_list_ = method.parameter_type_handle_list
            parameter_type_handle_list_[0] = type_handle_
            return_value_type_handle_ = method.return_value_type_handle

            add_present_method(type_handle_, method_name, TypeInferenceMethod(parameter_type_handle_list_, return_value_type_handle_), indent_level + 1)
    else:
        assert False, concrete_typing_constraints


# The `TypeVariable` and a NEW `TypeHandle` pointing to the `TypeVariable` are returned
# DO NOT set the concrete_typing_constraint of any `TypeHandle` in type_handle_iterable to the `TypeVariable`
def associate_with_type_variable(type_handle_iterable, indent_level=0):
    global type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict
    
    indent = '    ' * indent_level
    print(indent, f'associate_with_type_variable {type_handle_iterable}', file=sys.stderr)

    type_variable = TypeVariable()
    type_handle = TypeHandle()
    
    type_handle_set = set()
    for type_handle_ in type_handle_iterable:
        type_handle_set.update(recursively_iterate_associated_type_handles(type_handle_, indent_level + 1))

    type_handle_to_concrete_typing_constraints_dict[type_handle] = type_variable
    type_variable_to_type_handle_set_dict[type_variable] = type_handle_set
    
    return type_variable, type_handle


def update_type_handles_in_type_inference_method_according_to_function_definition(type_inference_method: TypeInferenceMethod, function_definition: FunctionDefinition, indent_level=0, *, is_method=False):
    indent = '    ' * indent_level
    print(indent, f'update_type_handles_in_type_inference_method_according_to_function_definition {type_inference_method} {function_definition}', file=sys.stderr)

    function_definition_parameter_type_annotation_list = function_definition.parameter_type_annotation_list
    function_definition_return_value_type_annotation = function_definition.return_value_type_annotation
    
    # Ignore initial 'self' or 'cls' parameter for methods
    for parameter_type_handle, parameter_type_annotation in zip(
        type_inference_method.parameter_type_handle_list[1:],
        function_definition_parameter_type_annotation_list[1:] if is_method else function_definition_parameter_type_annotation_list
    ):
        set_to_type_annotation(parameter_type_handle, parameter_type_annotation, indent_level + 1)

    set_to_type_annotation(type_inference_method.return_value_type_handle, function_definition_return_value_type_annotation, indent_level + 1)


def recursively_iterate_associated_type_handles(type_handle: TypeHandle, indent_level=0):
    global type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict
    
    indent = '    ' * indent_level
    print(indent, f'recursively_iterate_associated_type_handles {type_handle}', file=sys.stderr)
    
    concrete_typing_constraints = type_handle_to_concrete_typing_constraints_dict[type_handle]
    if isinstance(concrete_typing_constraints, TypeVariable):
        for type_handle_ in type_variable_to_type_handle_set_dict[concrete_typing_constraints]:
            yield from recursively_iterate_associated_type_handles(type_handle_, indent_level + 1)
    else:
        yield type_handle

We propose a set of typing rules for Numba IR expressions and describe how we generate typing constraints involving the presence of methods for variables we encounter based on the expressions they participate in.

## Typing Rules for Numba IR Expressions

### Unary and Binary Operations

We first propose typing rules for unary and binary operations, including arithmetic operations, comparisons, identity checks, and membership test operations. This covers the `unary`, `binop`, and `inplace_binop` expressions in Numba IR.

$$\frac{\Gamma \vdash e_1: T_1, T_1 = \mu \alpha . \{impl[unary]: \alpha \rightarrow T_2 \}}{\Gamma \vdash unary \: e_1: T_2}$$

$$\frac{\Gamma \vdash e_1: T_1, \Gamma \vdash e_2: T_2, T_1 = \mu \alpha . \{impl[binop]: \alpha \rightarrow T_2 \rightarrow T_3 \}}{\Gamma \vdash e_1 \: binop \: e_2: T_3}$$

In these typing rules, $unary$ represents a unary operation, and $binop$ represents a binary operation. At the same time, $impl$ is a dictionary that maps an operation to its implementation method called internally within the Python interpreter. For example, the implementation method of `+` is `__add__`, while the implementation method of `<` is `__lt__`.

### Iteration

The following typing rules are related to iteration. Before we start iteration, we have to get an *iterator* object from an *iterable* object (such as a Python `list`). This corresponds to the `getiter` expression in Numba IR and requires that the *iterable* object provides a method `__iter__`, as described in the typing rule below.

$$\frac{\Gamma \vdash e_1: T_1, T_1 = \mu \alpha . \{\_\_iter\_\_: \alpha \rightarrow T_2 \}}{\Gamma \vdash getiter \: e_1: T_2}$$

After getting an *iterator* object, we can take values from it. This is represented using the `iternext` expression in Numba IR and requires that the *iterator* provides a `__next__` method.

$$\frac{\Gamma \vdash e_2: T_2, T_2 = \mu \alpha . \{\_\_next\_\_: \alpha \rightarrow T_3 \}}{\Gamma \vdash iternext \: e_2: T_3}$$

### Indexing

Various types in Python can be indexed, including `list`'s and `dict`'s. In Numba IR, this is accomplished using the `getitem` or `static_getitem` expression and internally calls the `__getitem__` method of the type.

$$\frac{\Gamma \vdash e_1: T_1 \: \Gamma \vdash e_2: T_2 \: T_1 = \mu \alpha . \{\_\_getitem\_\_: \alpha \rightarrow T_2 \rightarrow T_3 \}}{\Gamma \vdash e_1[e_2]: T_3}$$

### Building `tuple`'s, `list`'s, `list`'s, and `dict`'s from Variables

In Numba IR, it is possible to construct `tuple`'s, `list`'s, `list`'s, and `dict`'s from variables using the `build_tuple`, `build_list`, `build_set`, and `build_map` expressions, respectively. The typing rules for these expressions are as follows.

$$\frac{\Gamma \vdash e_1: T \: \Gamma \vdash e_2: T \: \dots \: \Gamma \vdash e_n: T}{\Gamma \vdash build\_tuple \: e_1, e_2, \dots, e_n: tuple[T, \dots]}$$

$$\frac{\Gamma \vdash e_1: T \: \Gamma \vdash e_2: T \: \dots \: \Gamma \vdash e_n: T}{\Gamma \vdash build\_list \: e_1, e_2, \dots, e_n: list[T]}$$

$$\frac{\Gamma \vdash e_1: T \: \Gamma \vdash e_2: T \: \dots \: \Gamma \vdash e_n: T}{\Gamma \vdash build\_set \: e_1, e_2, \dots, e_n: set[T]}$$

$$\frac{\Gamma \vdash k_1: K \: \Gamma \vdash v_1: V \: \dots \: \Gamma \vdash k_n: K, \Gamma \vdash v_n: V}{\Gamma \vdash build\_map \: k_1, v_1, \dots, k_n, v_n: dict[K, V]}$$

In these typing rules, we use $tuple[T, \dots]$, $list[T]$, $set[T]$ to represent `tuple`'s, `list`'s, `list`'s containing elements of type $T$, and use $dict[K, V]$ to represent `dict`'s holding keys of type $K$ and values of type $V$. This notation is consistent with [Python's official type hinting generics in standard collections](https://peps.python.org/pep-0585).

### Calling a Callable Object

The `call` expression in Numba IR calls a *callable object*. In Python, a callable object is any object that could be called like a function, including global functions, methods, classes, and instances of classes that define the `__call__` method. In each case, the `__call__` method defined within the class of the callable object is invoked when the callable object is called, as described in the typing rule below.

$$\frac{\Gamma \vdash e_1: T_1 \: \Gamma \vdash e_2: T_2 \dots \: T_1 = \mu \alpha . \{\_\_call\_\_: \alpha \rightarrow T_2 \rightarrow \dots \rightarrow T_{n + 1} \}}{\Gamma \vdash e_1(e_2, \dots): T_{n + 1}}$$

For global functions which are instances of either `function` or `builtin_function_or_method`, this delegates to calling the global function itself, while for classes which are instances of `type`, this delegates to calling the class's constructor, which is either `__init__` or `__new__`.

### Phi Nodes

Numba IR is an IR in SSA form and supports Phi nodes, which select a value from multiple incoming values based on control flow. Since any selected value should be able to be used after the Phi node in the same way, we enforce the same type for each selected value in the following typing rule.

$$\frac{\Gamma \vdash e_1: T \: \Gamma \vdash e_2: T \dots \Gamma \vdash e_n: T}{\Gamma \vdash \text{phi}(e_1, e_2, \dots, e_n): T}$$

In [4]:
import operator


def handle_instruction(instruction, indent_level=0):
    indent = '    ' * indent_level
    print(indent, f'handle_instruction {instruction}', file=sys.stderr)

    if type(instruction) is Assign:
        handle_assign(instruction, indent_level + 1)
    elif type(instruction) is SetItem:
        handle_setitem(instruction, indent_level + 1)
    elif type(instruction) in (Return, Raise, StaticRaise, Jump, Branch, Del, TryRaise, StaticTryRaise, Print, EnterWith, PopBlock):
        pass
    else:
        assert False, f'cannot handle_instruction {instruction}'


def handle_variable_assignment(lhs_variable_name: str, rhs_variable_name: str, indent_level=0):
    global variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict

    indent = '    ' * indent_level
    print(indent, f'handle_variable_assignment {lhs_variable_name} {rhs_variable_name}', file=sys.stderr)

    # Distinguish between assigning a variable that already exists and assigning a fresh variable
    if lhs_variable_name in variable_name_to_type_handle_dict:
        lhs_variable_type_handle = get_type_handle_for_variable(lhs_variable_name, indent_level + 1)
        rhs_variable_type_handle = get_type_handle_for_variable(rhs_variable_name, indent_level + 1)

        new_lhs_variable_type_variable, new_lhs_variable_type_handle = associate_with_type_variable(
            [lhs_variable_type_handle, rhs_variable_type_handle],
            indent_level + 1
        )

        variable_name_to_type_handle_dict[lhs_variable_name] = new_lhs_variable_type_handle
    else:
        rhs_variable_type_handle = get_type_handle_for_variable(rhs_variable_name, indent_level + 1)

        variable_name_to_type_handle_dict[lhs_variable_name] = rhs_variable_type_handle


def handle_assign(instruction, indent_level=0):
    # <instruction.target: Var> = <instruction.value: AbstractRHS>
    indent = '    ' * indent_level
    print(indent, f'handle_assign {instruction}', file=sys.stderr)

    if type(instruction.value) is Arg:
        handle_assign_arg(instruction, indent_level + 1)
    elif type(instruction.value) is Const:
        handle_assign_const(instruction, indent_level + 1)
    elif type(instruction.value) is Global:
        handle_assign_global(instruction, indent_level + 1)
    elif type(instruction.value) is Var:
        handle_assign_var(instruction, indent_level + 1)
    elif type(instruction.value) is Expr:
        handle_assign_expr(instruction, indent_level + 1)
    else:
        assert False, f'cannot handle_assign {instruction}'


def handle_assign_arg(instruction, indent_level=0):
    # <instruction.target: Var> = arg(<instruction.value.index: int>, name=<instruction.value.name: str>)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_arg {instruction}', file=sys.stderr)

    get_type_handle_for_variable(instruction.target.name, indent_level + 1)


def handle_assign_const(instruction, indent_level=0):
    # <instruction.target: Var> = const(<type(instruction.value.value).__name__: str>, <instruction.value.value>)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_const {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    
    const = instruction.value.value
    concrete_class = ConcreteClass(module_name=type(const).__module__, class_name=type(const).__name__)
    
    set_to_type_annotation(instruction_target_type_handle, concrete_class, indent_level + 1)


def handle_assign_global(instruction, indent_level=0):
    # <instruction.target: Var> = global(<instruction.value.name: str>: <instruction.value.value>)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_global {instruction}', file=sys.stderr)
    
    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    
    if callable(instruction.value.value):
        if isinstance(instruction.value.value, type):
            concrete_class = ConcreteClass(module_name=instruction.value.value.__module__, class_name=instruction.value.value.__name__)
            set_to_type_annotation(instruction_target_type_handle, TypeInferenceConcreteClassObject(concrete_class), indent_level + 1)
        else:
            global_function = GlobalFunction(module_name=instruction.value.value.__module__, function_name=instruction.value.value.__name__)
            set_to_type_annotation(instruction_target_type_handle, TypeInferenceGlobalFunctionObject(global_function), indent_level + 1)
    else:
        assert False, f'cannot handle_assign_global {instruction}'


def handle_assign_var(instruction, indent_level=0):
    global variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict
    # <instruction.target: Var> = Var(<instruction.value.name: str>, <instruction.value.loc.short()>)

    indent = '    ' * indent_level
    print(indent, f'handle_assign_var {instruction}', file=sys.stderr)

    handle_variable_assignment(
        instruction.target.name,
        instruction.value.name,
        indent_level + 1
    )


def handle_assign_expr(instruction, indent_level=0):
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr {instruction}', file=sys.stderr)

    if instruction.value.op == 'call':
        handle_assign_expr_call(instruction, indent_level + 1)
    elif instruction.value.op == 'binop':
        handle_assign_expr_binop(instruction, indent_level + 1)
    elif instruction.value.op == 'inplace_binop':
        handle_assign_expr_inplace_binop(instruction, indent_level + 1)
    elif instruction.value.op == 'build_list':
        handle_assign_expr_build_list(instruction, indent_level + 1)
    elif instruction.value.op == 'getiter':
        handle_assign_expr_getiter(instruction, indent_level + 1)
    elif instruction.value.op == 'iternext':
        handle_assign_expr_iternext(instruction, indent_level + 1)
    elif instruction.value.op == 'pair_first':
        handle_assign_expr_pair_first(instruction, indent_level + 1)
    elif instruction.value.op == 'pair_second':
        handle_assign_expr_pair_second(instruction, indent_level + 1)
    elif instruction.value.op == 'getitem':
        handle_assign_expr_getitem(instruction, indent_level + 1)
    elif instruction.value.op == 'cast':
        handle_assign_expr_cast(instruction, indent_level + 1)
    elif instruction.value.op == 'phi':
        handle_assign_expr_phi(instruction, indent_level + 1)
    else:
        assert False, f'cannot handle_assign_expr {instruction}'


def handle_assign_expr_call(instruction, indent_level=0):
    # <instruction.target: Var> = call <instruction.value.func: Var>(args=<instruction.value.args: list>, kws=<instruction.value.kws: list>, vararg=<instruction.value.vararg>, varkwarg=<instruction.value.varkwarg>, target=<instruction.value.target>)
    # $20call_function_kw.9 = call $2load_global.0(args=[Var($const4.1, <ipython-input-10-d23331be2304>:2), Var($const6.2, <ipython-input-10-d23331be2304>:2), Var($const8.3, <ipython-input-10-d23331be2304>:2), Var($const10.4, <ipython-input-10-d23331be2304>:2), Var($const12.5, <ipython-input-10-d23331be2304>:2)], kws=[('a', Var($const14.6, <ipython-input-10-d23331be2304>:2)), ('b', Var($const16.7, <ipython-input-10-d23331be2304>:2))], vararg=None, varkwarg=None, target=None)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_call {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    
    instruction_value_func_type_handle = get_type_handle_for_variable(instruction.value.func.name, indent_level + 1)
    
    instruction_value_args_type_handle_list = [
        get_type_handle_for_variable(arg.name, indent_level + 1)
        for arg in instruction.value.args
    ]

    if instruction.value.kws or instruction.value.vararg or instruction.value.varkwarg:
        assert False, f'cannot handle_assign_expr_call {instruction}'
    
    add_present_method(
        instruction_value_func_type_handle,
        '__call__',
        TypeInferenceMethod(
            [instruction_value_func_type_handle] + instruction_value_args_type_handle_list,
            instruction_target_type_handle
        ),
        indent_level + 1
    )


# https://docs.python.org/3/library/operator.html#mapping-operators-to-functions
binary_operators_to_lhs_method_names = {
    # Assumed to return bool
    operator.is_: None,
    operator.is_not: None,
    operator.contains: '__contains__',
    # Unknown return value (e.g. for numpy.ndarray and another object)
    # https://docs.python.org/3/reference/datamodel.html#object.__lt__
    operator.lt: '__lt__',
    operator.le: '__le__',
    operator.eq: '__eq__',
    operator.ne: '__ne__',
    operator.gt: '__gt__',
    operator.ge: '__ge__',
    # Unknown return value (e.g. for numpy.ndarray and another object)
    # https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types
    operator.add: '__add__',
    operator.sub: '__sub__',
    operator.mul: '__mul__',
    operator.matmul: '__matmul__',
    operator.truediv: '__truediv__',
    operator.floordiv: '__floordiv__',
    operator.mod: '__mod__',
    operator.pow: '__pow__',
    operator.lshift: '__lshift__',
    operator.rshift: '__rshift__',
    operator.and_: '__and__',
    operator.xor: '__xor__',
    operator.or_: '__or__',
}

def handle_assign_expr_binop(instruction, indent_level=0):
    # <instruction.target: Var> = <instruction.value.fn: BuiltinFunctionType>(<instruction.value.lhs: Var>, <instruction.value.rhs: Var>)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_binop {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    instruction_value_lhs_type_handle = get_type_handle_for_variable(instruction.value.lhs.name, indent_level + 1)
    instruction_value_rhs_type_handle = get_type_handle_for_variable(instruction.value.rhs.name, indent_level + 1)

    instruction_value_lhs_method_name = binary_operators_to_lhs_method_names[instruction.value.fn]

    if instruction_value_lhs_method_name is not None:
        add_present_method(
            instruction_value_lhs_type_handle,
            instruction_value_lhs_method_name,
            TypeInferenceMethod(
                [instruction_value_lhs_type_handle, instruction_value_rhs_type_handle],
                instruction_target_type_handle
            ),
            indent_level + 1
        )
    
    if instruction.value.fn in (operator.is_, operator.is_not, operator.contains):
        set_to_type_annotation(instruction_target_type_handle, ConcreteClass(module_name='builtins', class_name='bool'), indent_level + 1)


def handle_assign_expr_inplace_binop(instruction, indent_level=0):
    # <instruction.target: Var> = inplace_binop(fn=<instruction.value.fn: BuiltinFunctionType>, immutable_fn=<instruction.value.immutable_fn: BuiltinFunctionType>, lhs=<instruction.value.lhs: Var>, rhs=<instruction.value.rhs: Var>)
    # inplace_binop(fn=<built-in function iadd>, immutable_fn=<built-in function add>, lhs=a, rhs=$const24.8, static_lhs=Undefined, static_rhs=Undefined)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_inplace_binop {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    instruction_value_lhs_type_handle = get_type_handle_for_variable(instruction.value.lhs.name, indent_level + 1)
    instruction_value_rhs_type_handle = get_type_handle_for_variable(instruction.value.rhs.name, indent_level + 1)
    
    instruction_value_lhs_method_name = binary_operators_to_lhs_method_names[instruction.value.immutable_fn]

    if instruction_value_lhs_method_name is not None:
        add_present_method(
            instruction_value_lhs_type_handle,
            instruction_value_lhs_method_name,
            TypeInferenceMethod(
                [instruction_value_lhs_type_handle, instruction_value_rhs_type_handle],
                instruction_target_type_handle
            ),
            indent_level + 1
        )
    
    if instruction.value.fn in (operator.is_, operator.is_not, operator.contains):
        set_to_type_annotation(instruction_target_type_handle, ConcreteClass(module_name='builtins', class_name='bool'), indent_level + 1)


def handle_assign_expr_build_list(instruction, indent_level=0):
    global type_variable_to_type_handle_set_dict
    # <instruction.target: Var> = build_list(items=<instruction.value.items: list>)
    # $8build_list.3 = build_list(items=[Var($const2.0, <ipython-input-25-997c60224512>:2), Var($const4.1, <ipython-input-25-997c60224512>:2), Var($const6.2, <ipython-input-25-997c60224512>:2)])
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_build_list {instruction}', file=sys.stderr)
    
    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    items_type_handle_list = [
        get_type_handle_for_variable(item.name, indent_level + 1)
        for item in instruction.value.items
    ]
    
    concrete_class = ConcreteClass(module_name='builtins', class_name='list')
    
    type_variable, _ = associate_with_type_variable(items_type_handle_list, indent_level + 1)
    
    set_to_type_annotation(instruction_target_type_handle, Subscription(concrete_class=concrete_class, type_annotation_tuple=(type_variable, )), indent_level + 1)


def handle_assign_expr_getiter(instruction, indent_level=0):
    # <instruction.target: Var> = getiter(value=<instruction.value.value: Var>)
    # $16get_iter.4 = getiter(value=$14call_function.3)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_getiter {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    instruction_value_value_type_handle = get_type_handle_for_variable(instruction.value.value.name, indent_level + 1)
    
    add_present_method(
        instruction_value_value_type_handle,
        '__iter__',
        TypeInferenceMethod(
            [instruction_value_value_type_handle],
            instruction_target_type_handle
        ),
        indent_level + 1
    )


def handle_assign_expr_iternext(instruction, indent_level=0):
    # <instruction.target: Var> = iternext(value=<instruction.value.value: Var>)
    # $26for_iter.1 = iternext(value=$phi26.0)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_iternext {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    instruction_value_value_type_handle = get_type_handle_for_variable(instruction.value.value.name, indent_level + 1)
    
    add_present_method(
        instruction_value_value_type_handle,
        '__next__',
        TypeInferenceMethod(
            [instruction_value_value_type_handle],
            instruction_target_type_handle
        ),
        indent_level + 1
    )


def handle_assign_expr_pair_first(instruction, indent_level=0):
    global variable_name_to_type_handle_dict
    # <instruction.target: Var> = pair_first(value=<instruction.value.value: Var>, attr=<instruction.value.attr: str>)
    # $26for_iter.2 = pair_first(value=$26for_iter.1)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_pair_first {instruction}', file=sys.stderr)

    handle_variable_assignment(
        instruction.target.name,
        instruction.value.value.name,
        indent_level + 1
    )


def handle_assign_expr_pair_second(instruction, indent_level=0):
    # <instruction.target: Var> = pair_second(value=<instruction.value.value: Var>, attr=<instruction.value.attr: str>)
    # $26for_iter.3 = pair_second(value=$26for_iter.1)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_pair_second {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    
    concrete_class = ConcreteClass(module_name='builtins', class_name='bool')
    
    set_to_type_annotation(instruction_target_type_handle, concrete_class, indent_level + 1)


def handle_assign_expr_getitem(instruction, indent_level=0):
    # <instruction.target: Var> = getitem(value=<instruction.value.value: Var>, index=<instruction.value.index: Var>, fn=<instruction.value.fn>)
    # $6binary_subscr.2 = getitem(value=x, index=$const4.1, fn=<built-in function getitem>)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_getitem {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    instruction_value_value_type_handle = get_type_handle_for_variable(instruction.value.value.name, indent_level + 1)
    instruction_value_index_type_handle = get_type_handle_for_variable(instruction.value.index.name, indent_level + 1)
    
    add_present_method(
        instruction_value_value_type_handle,
        '__getitem__',
        TypeInferenceMethod(
            [instruction_value_value_type_handle, instruction_value_index_type_handle],
            instruction_target_type_handle
        ),
        indent_level + 1
    )


def handle_assign_expr_cast(instruction, indent_level=0):
    global variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict
    # <instruction.target: Var> = cast(<instruction.value.value: Var>)
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_cast {instruction}', file=sys.stderr)

    handle_variable_assignment(
        instruction.target.name,
        instruction.value.value.name,
        indent_level + 1
    )


def handle_assign_expr_phi(instruction, indent_level=0):
    global type_variable_to_type_handle_set_dict
    
    # <instruction.target: Var> = phi(incoming_values=<instruction.value.incoming_values: list>, incoming_blocks=<instruction.value.incoming_blocks: list>)
    # j.2 = phi(incoming_values=[Var(j.1, <ipython-input-139-b39119991189>:4), Var(j, <ipython-input-139-b39119991189>:2)], incoming_blocks=[20, 6])
    indent = '    ' * indent_level
    print(indent, f'handle_assign_expr_phi {instruction}', file=sys.stderr)

    assert instruction.target.name not in variable_name_to_type_handle_dict

    incoming_values_type_handle_list = [
        get_type_handle_for_variable(incoming_value.name, indent_level + 1)
        for incoming_value in instruction.value.incoming_values
    ]
    
    type_variable, instruction_target_type_handle = associate_with_type_variable(incoming_values_type_handle_list, indent_level + 1)
    
    variable_name_to_type_handle_dict[instruction.target.name] = instruction_target_type_handle


def handle_setitem(instruction, indent_level=0):
    # <instruction.target: Var>[<instruction.index: Var>] = <instruction.value: Var>
    indent = '    ' * indent_level
    print(indent, f'handle_setitem {instruction}', file=sys.stderr)

    instruction_target_type_handle = get_type_handle_for_variable(instruction.target.name, indent_level + 1)
    instruction_index_type_handle = get_type_handle_for_variable(instruction.index.name, indent_level + 1)
    instruction_value_type_handle = get_type_handle_for_variable(instruction.value.name, indent_level + 1)

    return_value_type_handle = TypeHandle()
    type_handle_to_concrete_typing_constraints_dict[return_value_type_handle] = ConcreteClass(module_name='builtins', class_name='NoneType')
    
    add_present_method(
        instruction_target_type_handle,
        '__setitem__',
        TypeInferenceMethod(
            [instruction_target_type_handle, instruction_index_type_handle, instruction_value_type_handle],
            return_value_type_handle
        ),
        indent_level + 1
    )

##  Inferring Types for Variables from Typing Constraints

After we have generated the constraints, we resolve them to infer types for variables. Specifically, we determine which types each set of `TypeHandle`'s that should *resolve to the same type* can take, as well as which types the remaining `TypeHandle`'s can take.

We have mentioned that we associate each set of `TypeHandle`'s that should *resolve to the same type* with an external `TypeVariable`. However, there may be considerable overlap between the sets of `TypeHandle`'s covered by each external `TypeVariable`. Thus, we first calculate *disjoint sets* of `TypeHandle`'s that should *resolve to the same type*. Moreover, for each disjoint set, if it contains `TypeHandle`'s whose concrete typing constraints are type annotations, we calculate the *lowest subtype* of these type annotations and set every `TypeHandle` within the disjoint set to the *lowest subtype*. For example, if a disjoint set contains `TypeHandle`'s pointing to `object` and `int`, we should set all `TypeHandle`'s within the disjoint set to point to `int`.

In [5]:
import sys

from disjoint_set import DisjointSet
from more_itertools import pairwise

from subtyping import find_lowest_subtype_of_type_annotations_or_none
from type_annotation import *


def simplify_type_variables(indent_level=0):
    global type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

    indent = '    ' * indent_level
    print(indent, 'simplify_type_variables', file=sys.stderr)

    # Split into disjoint sets
    disjoint_sets = DisjointSet()

    for type_variable, type_handle_set in type_variable_to_type_handle_set_dict.items():
        if len(type_handle_set) > 1:
            for first, second in pairwise(type_handle_set):
                disjoint_sets.union(first, second)
        else:
            disjoint_sets.find(next(iter(type_handle_set)))

    # Associate each disjoint set with a new `TypeVariable`
    disjoint_set_index_to_new_type_variable_dict = dict()
    old_type_variable_to_new_type_variable_dict = dict()
    new_type_variable_to_type_handle_set_dict = dict()

    for type_variable, type_handle_set in type_variable_to_type_handle_set_dict.items():
        for i, disjoint_set in enumerate(
                disjoint_sets.itersets()
        ):
            if type_handle_set.issubset(disjoint_set):
                if i not in disjoint_set_index_to_new_type_variable_dict:
                    new_type_variable = TypeVariable()
                    disjoint_set_index_to_new_type_variable_dict[i] = new_type_variable
                    new_type_variable_to_type_handle_set_dict[new_type_variable] = disjoint_set
                old_type_variable_to_new_type_variable_dict[type_variable] = disjoint_set_index_to_new_type_variable_dict[i]

    print(indent, 'old_type_variable_to_new_type_variable_dict', old_type_variable_to_new_type_variable_dict, file=sys.stderr)

    # Replace each old `TypeVariable` with the corresponding new `TypeVariable`
    # Modifies `type_handle_to_concrete_typing_constraints_dict`, `type_variable_to_type_handle_set_dict`
    for type_handle in type_handle_to_concrete_typing_constraints_dict:
        concrete_typing_constraints = type_handle_to_concrete_typing_constraints_dict[type_handle]
        if is_type_annotation(concrete_typing_constraints):
            new_concrete_typing_constraints = replace_type_variables_in_type_annotation(
                concrete_typing_constraints,
                old_type_variable_to_new_type_variable_dict,
                indent_level + 1
            )
            type_handle_to_concrete_typing_constraints_dict[type_handle] = new_concrete_typing_constraints

    type_variable_to_type_handle_set_dict = new_type_variable_to_type_handle_set_dict

    # For each disjoint set, if it contains `TypeHandle`'s whose concrete typing constraints are type annotations,
    # We calculate the lowest subtype of these type annotations and set every `TypeHandle` within the disjoint set to the lowest subtype.
    lowest_subtype_of_type_annotations_or_none_list = []
    type_handle_to_resolve_set_list = []

    for type_variable, type_handle_set in type_variable_to_type_handle_set_dict.items():
        type_annotation_list = []
        type_handle_to_resolve_set = set()

        for type_handle in type_handle_set:
            concrete_typing_constraints = type_handle_to_concrete_typing_constraints_dict[type_handle]
            if is_type_annotation(concrete_typing_constraints):
                type_annotation_list.append(concrete_typing_constraints)
            elif isinstance(concrete_typing_constraints, (TypeInferenceClass, TypeInferenceUnknown)):
                type_handle_to_resolve_set.add(type_handle)
            else:
                assert False, f'unknown concrete_typing_constraints for {type_variable}: {concrete_typing_constraints}'

        lowest_subtype_of_type_annotations_or_none = find_lowest_subtype_of_type_annotations_or_none(type_annotation_list)

        lowest_subtype_of_type_annotations_or_none_list.append(lowest_subtype_of_type_annotations_or_none)
        type_handle_to_resolve_set_list.append(type_handle_to_resolve_set)

    type_variable_to_type_annotation_dict = dict()

    for i, type_variable, in enumerate(type_variable_to_type_handle_set_dict):
        lowest_subtype_of_type_annotations_or_none = lowest_subtype_of_type_annotations_or_none_list[i]
        type_handle_to_resolve_set = type_handle_to_resolve_set_list[i]
        if lowest_subtype_of_type_annotations_or_none is not None:
            type_variable_to_type_annotation_dict[type_variable] = lowest_subtype_of_type_annotations_or_none
        else:
            assert type_handle_to_resolve_set, 'if lowest_subtype_of_type_annotations_or_none is None, type_handle_to_resolve_set should not be empty'

    for type_variable, type_annotation in type_variable_to_type_annotation_dict.items():
        if type_handle in type_variable_to_type_handle_set_dict:
            for type_handle in type_variable_to_type_handle_set_dict[type_variable]:
                set_to_type_annotation(type_handle, type_annotation)

    for type_variable in type_variable_to_type_annotation_dict:
        type_variable_to_type_handle_set_dict.pop(type_variable, None)

    for type_handle_ in type_handle_to_concrete_typing_constraints_dict:
        concrete_typing_constraints_ = type_handle_to_concrete_typing_constraints_dict[type_handle_]
        if is_type_annotation(concrete_typing_constraints_):
            new_concrete_typing_constraints_ = replace_type_variables_in_type_annotation(
                concrete_typing_constraints_,
                type_variable_to_type_annotation_dict
            )
            type_handle_to_concrete_typing_constraints_dict[type_handle_] = new_concrete_typing_constraints_

Afterward, we are left with disjoint sets of `TypeHandle`'s whose concrete typing constraints are *not* type annotations but should still resolve to the same type, as well as any other remaining unresolved `TypeHandle`'s. We perform a *two-step* query to infer viable types for them.

The *first* query selects all types which feature all methods in the concrete typing constraints of the `TypeHandle`'s. This is an overapproximation, as the presence of a method does not necessarily mean that that method satisfies the typing constraints, as the type annotations of the parameters and the return value may not be consistent with our requirements. However, this allows us to rapidly prune a large search space of types that *definitely* do not satisfy the typing constraints, which is beneficial considering matching a type against our typing constraints incurs expensive structural subtyping calculations. For example, given that a variable has the methods `__ge__` and `__sub__`, the variable could possibly be an instance of the builtin types `frozenset`, `set`, `bool`, and `int`.

After selecting a set of *candidate* types via the first query, we go on with the *second* query, which further queries types that satisfy fine-grained typing constraints on methods.

In [6]:
def create_second_query(type_inference_class_list, indent_level=0):
    global type_handle_to_concrete_typing_constraints_dict

    indent = '    ' * indent_level
    print(indent, f'create_second_query {type_inference_class_list}', file=sys.stderr)

    second_query = dict()

    for type_inference_class in type_inference_class_list:
        for method_name, type_inference_method in type_inference_class.present_method_name_to_type_inference_method_dict.items():
            method_query = create_method_query(type_inference_method, indent_level + 1)

            if method_name not in second_query:
                second_query[method_name] = method_query
            else:
                second_query[method_name] = update_method_query(second_query[method_name], method_query, indent_level + 1)
    
    return second_query


def create_method_query(type_inference_method, indent_level=0):
    global type_handle_to_concrete_typing_constraints_dict

    indent = '    ' * indent_level
    print(indent, f'create_method_query {type_inference_method}', file=sys.stderr)

    method_query = dict()
    
    method_query["number_of_parameters"] = len(type_inference_method.parameter_type_handle_list)

    method_query["parameter_indices_to_parameter_constraints"] = parameter_indices_to_parameter_constraints = dict()

    # Skip the first parameter
    for i in range(1, len(type_inference_method.parameter_type_handle_list)):
        parameter_indices_to_parameter_constraints[i] = create_parameter_or_return_value_constraint(type_inference_method.parameter_type_handle_list[i], indent_level + 1)
    
    method_query["return_value_constraint"] = create_parameter_or_return_value_constraint(type_inference_method.return_value_type_handle, indent_level + 1)

    return method_query


def update_method_query(first_method_query, second_method_query, indent_level=0):
    global type_handle_to_concrete_typing_constraints_dict

    indent = '    ' * indent_level
    print(indent, f'update_method_query {first_method_query} {second_method_query}', file=sys.stderr)

    new_method_query = dict()

    new_method_query["number_of_parameters"] = number_of_parameters = max(
        first_method_query["number_of_parameters"],
        second_method_query["number_of_parameters"]
    )

    new_method_query["parameter_indices_to_parameter_constraints"] = parameter_indices_to_parameter_constraints = dict()

    # Contravariant for parameters
    # Skip the first parameter
    for i in range(1, number_of_parameters):
        first_method_query_parameter_constraint = first_method_query["parameter_indices_to_parameter_constraints"].get(i, None)
        second_method_query_parameter_constraint = second_method_query["parameter_indices_to_parameter_constraints"].get(i, None)
        
        if first_method_query_parameter_constraint is None and second_method_query_parameter_constraint is None:
            new_method_query_parameter_constraint = None
        elif first_method_query_parameter_constraint is not None and second_method_query_parameter_constraint is None:
            new_method_query_parameter_constraint = first_method_query_parameter_constraint
        elif first_method_query_parameter_constraint is None and second_method_query_parameter_constraint is not None:
            new_method_query_parameter_constraint = second_method_query_parameter_constraint
        else:
            if type_annotation_subtyping(
                first_method_query_parameter_constraint,
                second_method_query_parameter_constraint,
                indent_level + 1
            )[0]:
                new_method_query_parameter_constraint = second_method_query_parameter_constraint
            elif type_annotation_subtyping(
                second_method_query_parameter_constraint,
                first_method_query_parameter_constraint,
                indent_level + 1
            )[0]:
                new_method_query_parameter_constraint = first_method_query_parameter_constraint
            else:
                assert False, f'{first_method_query_parameter_constraint} and {second_method_query_parameter_constraint} are unrelated'
        
        parameter_indices_to_parameter_constraints[i] = new_method_query_parameter_constraint

    # Covariant for return value
    first_method_query_return_value_constraint = first_method_query["return_value_constraint"]
    second_method_query_return_value_constraint = second_method_query["return_value_constraint"]

    if first_method_query_return_value_constraint is None and second_method_query_return_value_constraint is None:
        new_method_query_return_value_constraint = None
    elif first_method_query_return_value_constraint is not None and second_method_query_return_value_constraint is None:
        new_method_query_return_value_constraint = first_method_query_return_value_constraint
    elif first_method_query_return_value_constraint is None and second_method_query_return_value_constraint is not None:
        new_method_query_return_value_constraint = second_method_query_return_value_constraint
    else:
        if type_annotation_subtyping(
            first_method_query_return_value_constraint,
            second_method_query_return_value_constraint,
            indent_level + 1
        )[0]:
            new_method_query_return_value_constraint = first_method_query_return_value_constraint
        elif type_annotation_subtyping(
            second_method_query_return_value_constraint,
            first_method_query_return_value_constraint,
            indent_level + 1
        )[0]:
            new_method_query_return_value_constraint = second_method_query_return_value_constraint
        else:
            assert False, f'{first_method_query_return_value_constraint} and {second_method_query_return_value_constraint} are unrelated'
    
    new_method_query["return_value_constraint"] = new_method_query_return_value_constraint

    return new_method_query


def create_parameter_or_return_value_constraint(parameter_or_return_value_type_handle, indent_level=0):
    global type_handle_to_concrete_typing_constraints_dict

    indent = '    ' * indent_level
    print(indent, f'create_parameter_or_return_value_constraint {parameter_or_return_value_type_handle}', file=sys.stderr)

    implementation = type_handle_to_concrete_typing_constraints_dict[parameter_or_return_value_type_handle]

    if isinstance(implementation, (TypeVariable, ConcreteClass, Subscription, list)):
        return implementation
    else:
        return None

In [7]:
from subtyping import *


def execute_second_query(second_query, concrete_class_list, indent_level=0):
    indent = '    ' * indent_level
    print(indent, f'execute_second_query {second_query} {concrete_class_list}', file=sys.stderr)

    all_satisfying_concrete_classes = dict()

    for concrete_class in concrete_class_list:
        satisfying_concrete_class = create_satisfying_concrete_class(second_query, concrete_class, indent_level + 1)
        if satisfying_concrete_class is not None:
            all_satisfying_concrete_classes[concrete_class] = satisfying_concrete_class
            print(indent, f'{concrete_class} added to all_satisfying_concrete_classes', file=sys.stderr)
        else:
            print(indent, f'{concrete_class} not added to all_satisfying_concrete_classes', file=sys.stderr)
    
    return all_satisfying_concrete_classes


def create_satisfying_concrete_class(second_query, concrete_class, indent_level=0):
    indent = '    ' * indent_level
    print(indent, f'create_satisfying_concrete_class {second_query} {concrete_class}', file=sys.stderr)

    class_definition = look_up_class(concrete_class, indent_level + 1)

    type_variable_to_type_annotation_list_dict = {
        type_variable: list()
        for type_variable in class_definition.type_variable_list
    }
    
    # Check class_definition has all methods
    if not all((
        method_name in class_definition.method_name_to_method_list_dict
        for method_name in second_query
    )):
        print(indent, f'{concrete_class} does not have all methods', file=sys.stderr)
        return None
    
    satisfying_concrete_class = dict()

    # Try to create an AllSatisfyingMethods object for each method
    for method_name, method_query in second_query.items():
        all_satisfying_methods = create_all_satisfying_methods(method_query, class_definition.method_name_to_method_list_dict[method_name], type_variable_to_type_annotation_list_dict, indent_level + 1)
        if all_satisfying_methods:
            satisfying_concrete_class[method_name] = all_satisfying_methods
            print(indent, f'{all_satisfying_methods} created for {method_name}', file=sys.stderr)
        else:
            print(indent, f'cannot create an AllSatisfyingMethods object for {method_name}', file=sys.stderr)
            return None

    type_variable_to_type_annotation_dict = dict()

    for type_variable, type_annotation_list in type_variable_to_type_annotation_list_dict.items():
        if type_annotation_list:
            type_annotation = find_lowest_subtype_of_type_annotations_or_none(type_annotation_list)
        else:
            type_annotation = type_variable
        
        type_variable_to_type_annotation_dict[type_variable] = type_annotation

    satisfying_concrete_class['type_variable_to_type_annotation_dict'] = type_variable_to_type_annotation_dict

    return satisfying_concrete_class


def create_all_satisfying_methods(method_query, method_list, type_variable_to_type_annotation_list_dict, indent_level=0):
    indent = '    ' * indent_level
    print(indent, f'create_all_satisfying_methods {method_query} {method_list} {type_variable_to_type_annotation_list_dict}', file=sys.stderr)

    all_satisfying_methods = dict()

    for (i, method) in enumerate(method_list):
        satisfying_method = create_satisfying_method(method_query, method, type_variable_to_type_annotation_list_dict, indent_level + 1)
        if satisfying_method is not None:
            all_satisfying_methods[i] = satisfying_method
            print(indent, f'{satisfying_method} added to all_satisfying_methods', file=sys.stderr)
        else:
            print(indent, f'{satisfying_method} not added to all_satisfying_methods', file=sys.stderr)
    
    return all_satisfying_methods


def create_satisfying_method(method_query, method, type_variable_to_type_annotation_list_dict, indent_level=0):
    indent = '    ' * indent_level
    print(indent, f'create_satisfying_method {method_query} {method} {type_variable_to_type_annotation_list_dict}', file=sys.stderr)

    if len(method.parameter_type_annotation_list) < method_query["number_of_parameters"]:
        return None
    
    satisfying_method = dict()
    satisfying_method["parameter_type_annotations"] = dict()
    
    for parameter_index, parameter_constraint in method_query["parameter_indices_to_parameter_constraints"].items():
        parameter_type_annotation = create_parameter_or_return_value_type_annotation(
            parameter_constraint,
            method.parameter_type_annotation_list[parameter_index],
            type_variable_to_type_annotation_list_dict,
            indent_level + 1
        )
        if parameter_type_annotation is not None:
            satisfying_method["parameter_type_annotations"][parameter_index] = parameter_type_annotation
        else:
            return None
    
    return_value_constraint = method_query["return_value_constraint"]
    return_value_type_annotation = create_parameter_or_return_value_type_annotation(
        return_value_constraint,
        method.return_value_type_annotation,
        type_variable_to_type_annotation_list_dict,
        indent_level + 1
    )
    
    if return_value_type_annotation is not None:
        satisfying_method["return_value_type_annotation"] = return_value_type_annotation
    else:
        return None
    
    return satisfying_method


def create_parameter_or_return_value_type_annotation(constraint, type_annotation, type_variable_to_type_annotation_list_dict, indent_level=0):
    indent = '    ' * indent_level
    print(indent, f'create_parameter_or_return_value_type_annotation {constraint} {type_annotation} {type_variable_to_type_annotation_list_dict}', file=sys.stderr)
    
    if constraint is None:
        return { "type_annotation": type_annotation, "relation": "subtype" }
    else:
        if type_annotation_subtyping(constraint, type_annotation)[0]:
            type_variable_subtyping_digraph = type_annotation_subtyping(constraint, type_annotation)[1]
            for type_variable in iterate_type_variables_in_type_annotation(
                type_annotation,
                indent_level + 1
            ):
                if type_variable in type_variable_to_type_annotation_list_dict and type_variable in type_variable_subtyping_digraph:
                    type_variable_to_type_annotation_list_dict[type_variable].extend(type_variable_subtyping_digraph.predecessors(type_variable))

            return { "type_annotation": type_annotation, "relation": "supertype" }
        elif type_annotation_subtyping(type_annotation, constraint)[0]:
            return { "type_annotation": type_annotation, "relation": "subtype" }
        else:
            return None

# Running Example

In [8]:
def shell_sort(collection):
    # Marcin Ciura's gap sequence
    gaps = [701, 301, 132, 57, 23, 10, 4, 1]
    for gap in gaps:
        i = gap
        while i < len(collection):
            temp = collection[i]
            j = i
            while j >= gap and collection[j - gap] > temp:
                collection[j] = collection[j - gap]
                j -= gap
            collection[j] = temp
            i += 1

## Generating Typing Constraints for Variables

In [9]:
from numba_ssa_ir import *

In [10]:
from numba_ssa_ir_instructions import *

In [11]:
ir = numba_ssa_ir(shell_sort)

In [12]:
for instruction in numba_ssa_ir_instructions(ir):
    print(instruction)

collection = arg(0, name=collection)
$const2.0 = const(int, 701)
$const4.1 = const(int, 301)
$const6.2 = const(int, 132)
$const8.3 = const(int, 57)
$const10.4 = const(int, 23)
$const12.5 = const(int, 10)
$const14.6 = const(int, 4)
$const16.7 = const(int, 1)
gaps = build_list(items=[Var($const2.0, 861065169.py:3), Var($const4.1, 861065169.py:3), Var($const6.2, 861065169.py:3), Var($const8.3, 861065169.py:3), Var($const10.4, 861065169.py:3), Var($const12.5, 861065169.py:3), Var($const14.6, 861065169.py:3), Var($const16.7, 861065169.py:3)])
$24get_iter.10 = getiter(value=gaps)
$phi26.0 = $24get_iter.10
jump 26
$26for_iter.1 = iternext(value=$phi26.0)
$26for_iter.2 = pair_first(value=$26for_iter.1)
$26for_iter.3 = pair_second(value=$26for_iter.1)
$phi28.1 = $26for_iter.2
branch $26for_iter.3, 28, 128
gap = $phi28.1
i = gap
jump 34
i.2 = phi(incoming_values=[Var(i.1, 861065169.py:13), Var(i, 861065169.py:5)], incoming_blocks=[108, 28])
$36load_global.2 = global(len: <built-in function len>)

In [13]:
print(numba_ssa_ir_instructions(ir)[0])

collection = arg(0, name=collection)


In [14]:
handle_instruction(numba_ssa_ir_instructions(ir)[0])

 handle_instruction collection = arg(0, name=collection)
     handle_assign collection = arg(0, name=collection)
         handle_assign_arg collection = arg(0, name=collection)
             get_type_handle_for_variable collection


In [15]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None},
 {})

In [16]:
for instruction in numba_ssa_ir_instructions(ir)[1:9]:
    print(instruction)

$const2.0 = const(int, 701)
$const4.1 = const(int, 301)
$const6.2 = const(int, 132)
$const8.3 = const(int, 57)
$const10.4 = const(int, 23)
$const12.5 = const(int, 10)
$const14.6 = const(int, 4)
$const16.7 = const(int, 1)


In [17]:
for instruction in numba_ssa_ir_instructions(ir)[1:9]:
    handle_instruction(instruction)

 handle_instruction $const2.0 = const(int, 701)
     handle_assign $const2.0 = const(int, 701)
         handle_assign_const $const2.0 = const(int, 701)
             get_type_handle_for_variable $const2.0
             set_to_type_annotation <__main__.TypeHandle object at 0x7fcd75dd9250> ConcreteClass(module_name='builtins', class_name='int')
             None -> ConcreteClass(module_name='builtins', class_name='int')
 handle_instruction $const4.1 = const(int, 301)
     handle_assign $const4.1 = const(int, 301)
         handle_assign_const $const4.1 = const(int, 301)
             get_type_handle_for_variable $const4.1
             set_to_type_annotation <__main__.TypeHandle object at 0x7fcd75dd9310> ConcreteClass(module_name='builtins', class_name='int')
             None -> ConcreteClass(module_name='builtins', class_name='int')
 handle_instruction $const6.2 = const(int, 132)
     handle_assign $const6.2 = const(int, 132)
         handle_assign_const $const6.2 = const(int, 132)
        

In [18]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None,
  <__main__.TypeHandle at 0x7fcd75dd9250>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9310>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9340>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9370>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd93d0>

In [19]:
print(numba_ssa_ir_instructions(ir)[9])

gaps = build_list(items=[Var($const2.0, 861065169.py:3), Var($const4.1, 861065169.py:3), Var($const6.2, 861065169.py:3), Var($const8.3, 861065169.py:3), Var($const10.4, 861065169.py:3), Var($const12.5, 861065169.py:3), Var($const14.6, 861065169.py:3), Var($const16.7, 861065169.py:3)])


In [20]:
handle_instruction(numba_ssa_ir_instructions(ir)[9])

 handle_instruction gaps = build_list(items=[Var($const2.0, 861065169.py:3), Var($const4.1, 861065169.py:3), Var($const6.2, 861065169.py:3), Var($const8.3, 861065169.py:3), Var($const10.4, 861065169.py:3), Var($const12.5, 861065169.py:3), Var($const14.6, 861065169.py:3), Var($const16.7, 861065169.py:3)])
     handle_assign gaps = build_list(items=[Var($const2.0, 861065169.py:3), Var($const4.1, 861065169.py:3), Var($const6.2, 861065169.py:3), Var($const8.3, 861065169.py:3), Var($const10.4, 861065169.py:3), Var($const12.5, 861065169.py:3), Var($const14.6, 861065169.py:3), Var($const16.7, 861065169.py:3)])
         handle_assign_expr gaps = build_list(items=[Var($const2.0, 861065169.py:3), Var($const4.1, 861065169.py:3), Var($const6.2, 861065169.py:3), Var($const8.3, 861065169.py:3), Var($const10.4, 861065169.py:3), Var($const12.5, 861065169.py:3), Var($const14.6, 861065169.py:3), Var($const16.7, 861065169.py:3)])
             handle_assign_expr_build_list gaps = build_list(items=[Var($co

In [21]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None,
  <__main__.TypeHandle at 0x7fcd75dd9250>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9310>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9340>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9370>: ConcreteClass(module_name='builtins', class_nam

In [22]:
print(numba_ssa_ir_instructions(ir)[10])

$24get_iter.10 = getiter(value=gaps)


In [23]:
handle_instruction(numba_ssa_ir_instructions(ir)[10])

 handle_instruction $24get_iter.10 = getiter(value=gaps)
     handle_assign $24get_iter.10 = getiter(value=gaps)
         handle_assign_expr $24get_iter.10 = getiter(value=gaps)
             handle_assign_expr_getiter $24get_iter.10 = getiter(value=gaps)
                 get_type_handle_for_variable $24get_iter.10
                 get_type_handle_for_variable gaps
                 add_present_method <__main__.TypeHandle object at 0x7fcd75dd96d0> __iter__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75dd96d0>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd75dcb250>)
                 concrete_typing_constraints: Subscription(concrete_class=ConcreteClass(module_name='builtins', class_name='list'), type_annotation_tuple=(<type_annotation.TypeVariable object at 0x7fcd75dd9670>,))
 look_up_class ConcreteClass(module_name='builtins', class_name='list')
 cache miss
     parse_class ConcreteClass(module_name='builtins', class_name='list')

                                     parse_node_to_type_annotation typing Name(id='_T_co', ctx=Load())
                                         look_up_name typing _T_co <function look_up_name at 0x7fcdb80b1dc0>
                                         cache hit
                                 iterate_type_variables_in_type_annotation <type_annotation.TypeVariable object at 0x7fcd90e7b670>
                             looking up base class ConcreteClass(module_name='typing', class_name='Container')
                                 look_up_class ConcreteClass(module_name='typing', class_name='Container')
                                 cache miss
                                     parse_class ConcreteClass(module_name='typing', class_name='Container') dict_keys(['__contains__'])
                                         parse_node_to_type_annotation typing Subscript(value=Name(id='Protocol', ctx=Load()), slice=Index(value=Name(id='_T_co', ctx=Load())), ctx=Load())
                   

                             parse_class ConcreteClass(module_name='typing', class_name='Reversible') dict_keys(['__reversed__'])
                                 parse_node_to_type_annotation typing Subscript(value=Name(id='Iterable', ctx=Load()), slice=Index(value=Name(id='_T_co', ctx=Load())), ctx=Load())
                                     parse_node_to_type_annotation typing Name(id='Iterable', ctx=Load())
                                         look_up_name typing Iterable <function look_up_name at 0x7fcdb80b1dc0>
                                         cache hit
                                     parse_node_to_type_annotation typing Name(id='_T_co', ctx=Load())
                                         look_up_name typing _T_co <function look_up_name at 0x7fcdb80b1dc0>
                                         cache hit
                                 iterate_type_variables_in_type_annotation <type_annotation.TypeVariable object at 0x7fcd90e7b670>
                           

                     instantiating type variables in base class ConcreteClass(module_name='typing', class_name='Generic') with [<type_annotation.TypeVariable object at 0x7fcd90e7b670>]
                         instantiate_type_variables_in_class_definition ClassDefinition(type_variable_list=[], method_name_to_method_list_dict={}, staticmethod_name_to_staticmethod_list_dict={}, property_name_to_property_type_annotation_dict={}) [<type_annotation.TypeVariable object at 0x7fcd90e7b670>]
                     adding all methods to method_name_to_method_list_dict
                     adding all staticmethods to staticmethod_name_to_staticmethod_list_dict
                     adding all properties to property_name_to_property_type_annotation_dict
                         parse_method ConcreteClass(module_name='typing', class_name='Sequence') __getitem__
                             parse_node_to_type_annotation typing Name(id='int', ctx=Load())
                                 look_up_name ty

                             look_up_name _collections_abc Iterable <function look_up_name at 0x7fcdb80b1dc0>
                             cache miss
                                 look_up_name typing Iterable <function look_up_name at 0x7fcdb80b1dc0>
                                 cache hit
                 parse_node_to_type_annotation builtins Name(id='_T', ctx=Load())
                     look_up_name builtins _T <function look_up_name at 0x7fcdb80b1dc0>
                     cache hit
             iterate_type_variables_in_type_annotation Subscription(concrete_class=ConcreteClass(module_name='typing', class_name='Iterable'), type_annotation_tuple=(<type_annotation.TypeVariable object at 0x7fcd9136bb20>,))
                 iterate_type_variables_in_type_annotation <type_annotation.TypeVariable object at 0x7fcd9136bb20>
             parse_node_to_type_annotation builtins Constant(value=None, kind=None)
             iterate_type_variables_in_type_annotation ConcreteClass(module_na

                                     parse_node_to_type_annotation typing Constant(value=Ellipsis, kind=None)
                 parse_node_to_type_annotation builtins Tuple(elts=[List(elts=[Name(id='_T', ctx=Load())], ctx=Load()), Name(id='SupportsRichComparison', ctx=Load())], ctx=Load())
                     parse_node_to_type_annotation builtins List(elts=[Name(id='_T', ctx=Load())], ctx=Load())
                         parse_node_to_type_annotation builtins Name(id='_T', ctx=Load())
                             look_up_name builtins _T <function look_up_name at 0x7fcdb80b1dc0>
                             cache hit
                     parse_node_to_type_annotation builtins Name(id='SupportsRichComparison', ctx=Load())
                         look_up_name builtins SupportsRichComparison <function look_up_name at 0x7fcdb80b1dc0>
                         cache miss
                             look_up_name _typeshed SupportsRichComparison <function look_up_name at 0x7fcdb80b1dc0>
   

In [24]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None,
  <__main__.TypeHandle at 0x7fcd75dd9250>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9310>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9340>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7

In [25]:
for instruction in numba_ssa_ir_instructions(ir)[11:18]:
    print(instruction)

$phi26.0 = $24get_iter.10
jump 26
$26for_iter.1 = iternext(value=$phi26.0)
$26for_iter.2 = pair_first(value=$26for_iter.1)
$26for_iter.3 = pair_second(value=$26for_iter.1)
$phi28.1 = $26for_iter.2
branch $26for_iter.3, 28, 128


In [26]:
handle_instruction(numba_ssa_ir_instructions(ir)[11])

 handle_instruction $phi26.0 = $24get_iter.10
     handle_assign $phi26.0 = $24get_iter.10
         handle_assign_var $phi26.0 = $24get_iter.10
             handle_variable_assignment $phi26.0 $24get_iter.10
                 get_type_handle_for_variable $24get_iter.10


In [27]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None,
  <__main__.TypeHandle at 0x7fcd75dd9250>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9310>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9340>: ConcreteClass(module_name='buil

In [28]:
print(numba_ssa_ir_instructions(ir)[13])

$26for_iter.1 = iternext(value=$phi26.0)


In [29]:
handle_instruction(numba_ssa_ir_instructions(ir)[13])

 handle_instruction $26for_iter.1 = iternext(value=$phi26.0)
     handle_assign $26for_iter.1 = iternext(value=$phi26.0)
         handle_assign_expr $26for_iter.1 = iternext(value=$phi26.0)
             handle_assign_expr_iternext $26for_iter.1 = iternext(value=$phi26.0)
                 get_type_handle_for_variable $26for_iter.1
                 get_type_handle_for_variable $phi26.0
                 add_present_method <__main__.TypeHandle object at 0x7fcd75dcb250> __next__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75dcb250>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90ade490>)
                 concrete_typing_constraints: Subscription(concrete_class=ConcreteClass(module_name='typing', class_name='Iterator'), type_annotation_tuple=(<type_annotation.TypeVariable object at 0x7fcd75dd9670>,))
 look_up_class ConcreteClass(module_name='typing', class_name='Iterator')
 cache miss
     parse_class ConcreteClass(module_name='typin

In [30]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None,
  <__main__.TypeHandle at 0x7fcd75dd9250>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9310>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.Ty

In [31]:
print(numba_ssa_ir_instructions(ir)[14])

$26for_iter.2 = pair_first(value=$26for_iter.1)


In [32]:
handle_instruction(numba_ssa_ir_instructions(ir)[14])

 handle_instruction $26for_iter.2 = pair_first(value=$26for_iter.1)
     handle_assign $26for_iter.2 = pair_first(value=$26for_iter.1)
         handle_assign_expr $26for_iter.2 = pair_first(value=$26for_iter.1)
             handle_assign_expr_pair_first $26for_iter.2 = pair_first(value=$26for_iter.1)
                 handle_variable_assignment $26for_iter.2 $26for_iter.1
                     get_type_handle_for_variable $26for_iter.1


In [33]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None,
  <__main__.TypeHandle at 0x7fcd75dd9250>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75dd9310>: ConcreteCla

In [34]:
print(numba_ssa_ir_instructions(ir)[15])

$26for_iter.3 = pair_second(value=$26for_iter.1)


In [35]:
handle_instruction(numba_ssa_ir_instructions(ir)[15])

 handle_instruction $26for_iter.3 = pair_second(value=$26for_iter.1)
     handle_assign $26for_iter.3 = pair_second(value=$26for_iter.1)
         handle_assign_expr $26for_iter.3 = pair_second(value=$26for_iter.1)
             handle_assign_expr_pair_second $26for_iter.3 = pair_second(value=$26for_iter.1)
                 get_type_handle_for_variable $26for_iter.3
                 set_to_type_annotation <__main__.TypeHandle object at 0x7fcd90bfd1c0> ConcreteClass(module_name='builtins', class_name='bool')
                 None -> ConcreteClass(module_name='builtins', class_name='bool')


In [36]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None,
  <__main__.TypeHandle at 0x7fcd75dd9250>: ConcreteClass(module_name='builtins', class_name='i

In [37]:
print(numba_ssa_ir_instructions(ir)[16])

$phi28.1 = $26for_iter.2


In [38]:
handle_instruction(numba_ssa_ir_instructions(ir)[16])

 handle_instruction $phi28.1 = $26for_iter.2
     handle_assign $phi28.1 = $26for_iter.2
         handle_assign_var $phi28.1 = $26for_iter.2
             handle_variable_assignment $phi28.1 $26for_iter.2
                 get_type_handle_for_variable $26for_iter.2


In [39]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>},
 {<__main__.TypeHandle at 0x7fcd75e0c3a0>: None,
  <__main__.TypeHandle at 0x7fcd75dd925

In [40]:
print(numba_ssa_ir_instructions(ir)[17])

branch $26for_iter.3, 28, 128


In [41]:
print(numba_ssa_ir_instructions(ir)[18])

gap = $phi28.1


In [42]:
handle_instruction(numba_ssa_ir_instructions(ir)[18])

 handle_instruction gap = $phi28.1
     handle_assign gap = $phi28.1
         handle_assign_var gap = $phi28.1
             handle_variable_assignment gap $phi28.1
                 get_type_handle_for_variable $phi28.1


In [43]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>},
 {<__main__.TypeHandle at 0x7fcd75e0c3

In [44]:
print(numba_ssa_ir_instructions(ir)[19])

i = gap


In [45]:
handle_instruction(numba_ssa_ir_instructions(ir)[19])

 handle_instruction i = gap
     handle_assign i = gap
         handle_assign_var i = gap
             handle_variable_assignment i gap
                 get_type_handle_for_variable gap


In [46]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [47]:
print(numba_ssa_ir_instructions(ir)[20])

jump 34


In [48]:
print(numba_ssa_ir_instructions(ir)[21])

i.2 = phi(incoming_values=[Var(i.1, 861065169.py:13), Var(i, 861065169.py:5)], incoming_blocks=[108, 28])


In [49]:
handle_instruction(numba_ssa_ir_instructions(ir)[21])

 handle_instruction i.2 = phi(incoming_values=[Var(i.1, 861065169.py:13), Var(i, 861065169.py:5)], incoming_blocks=[108, 28])
     handle_assign i.2 = phi(incoming_values=[Var(i.1, 861065169.py:13), Var(i, 861065169.py:5)], incoming_blocks=[108, 28])
         handle_assign_expr i.2 = phi(incoming_values=[Var(i.1, 861065169.py:13), Var(i, 861065169.py:5)], incoming_blocks=[108, 28])
             handle_assign_expr_phi i.2 = phi(incoming_values=[Var(i.1, 861065169.py:13), Var(i, 861065169.py:5)], incoming_blocks=[108, 28])
                 get_type_handle_for_variable i.1
                 get_type_handle_for_variable i
                 associate_with_type_variable [<__main__.TypeHandle object at 0x7fcd910b0d60>, <__main__.TypeHandle object at 0x7fcd90ade490>]
                     recursively_iterate_associated_type_handles <__main__.TypeHandle object at 0x7fcd910b0d60>
                     recursively_iterate_associated_type_handles <__main__.TypeHandle object at 0x7fcd90ade490>
        

In [50]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [51]:
print(numba_ssa_ir_instructions(ir)[22])

$36load_global.2 = global(len: <built-in function len>)


In [52]:
handle_instruction(numba_ssa_ir_instructions(ir)[22])

 handle_instruction $36load_global.2 = global(len: <built-in function len>)
     handle_assign $36load_global.2 = global(len: <built-in function len>)
         handle_assign_global $36load_global.2 = global(len: <built-in function len>)
             get_type_handle_for_variable $36load_global.2
             set_to_type_annotation <__main__.TypeHandle object at 0x7fcd910b0850> TypeInferenceGlobalFunctionObject(global_function=GlobalFunction(module_name='builtins', function_name='len'))
             None -> TypeInferenceGlobalFunctionObject(global_function=GlobalFunction(module_name='builtins', function_name='len'))


In [53]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [54]:
print(numba_ssa_ir_instructions(ir)[23])

$40call_function.4 = call $36load_global.2(collection, func=$36load_global.2, args=[Var(collection, 861065169.py:3)], kws=(), vararg=None, varkwarg=None, target=None)


In [55]:
handle_instruction(numba_ssa_ir_instructions(ir)[23])

 handle_instruction $40call_function.4 = call $36load_global.2(collection, func=$36load_global.2, args=[Var(collection, 861065169.py:3)], kws=(), vararg=None, varkwarg=None, target=None)
     handle_assign $40call_function.4 = call $36load_global.2(collection, func=$36load_global.2, args=[Var(collection, 861065169.py:3)], kws=(), vararg=None, varkwarg=None, target=None)
         handle_assign_expr $40call_function.4 = call $36load_global.2(collection, func=$36load_global.2, args=[Var(collection, 861065169.py:3)], kws=(), vararg=None, varkwarg=None, target=None)
             handle_assign_expr_call $40call_function.4 = call $36load_global.2(collection, func=$36load_global.2, args=[Var(collection, 861065169.py:3)], kws=(), vararg=None, varkwarg=None, target=None)
                 get_type_handle_for_variable $40call_function.4
                 get_type_handle_for_variable $36load_global.2
                 get_type_handle_for_variable collection
                 add_present_method <__main

In [56]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [57]:
print(numba_ssa_ir_instructions(ir)[24])

$42compare_op.5 = i.2 < $40call_function.4


In [58]:
handle_instruction(numba_ssa_ir_instructions(ir)[24])

 handle_instruction $42compare_op.5 = i.2 < $40call_function.4
     handle_assign $42compare_op.5 = i.2 < $40call_function.4
         handle_assign_expr $42compare_op.5 = i.2 < $40call_function.4
             handle_assign_expr_binop $42compare_op.5 = i.2 < $40call_function.4
                 get_type_handle_for_variable $42compare_op.5
                 get_type_handle_for_variable i.2
                 get_type_handle_for_variable $40call_function.4
                 add_present_method <__main__.TypeHandle object at 0x7fcd910b0370> __lt__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd910b0370>, <__main__.TypeHandle object at 0x7fcd912442b0>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90ab8310>)
                 concrete_typing_constraints: <type_annotation.TypeVariable object at 0x7fcd910b0250>
                     add_present_method <__main__.TypeHandle object at 0x7fcd75dd9400> __lt__ TypeInferenceMethod(parameter_type_handle_

                                                                         parse_node_to_type_annotation _typeshed Name(id='memoryview', ctx=Load())
                                                                             look_up_name _typeshed memoryview <function look_up_name at 0x7fcdb80b1dc0>
                                                                             cache miss
                                                                                 look_up_name builtins memoryview <function look_up_name at 0x7fcdb80b1dc0>
                                                                                 cache miss
                                                                     parse_node_to_type_annotation _typeshed Subscript(value=Attribute(value=Name(id='array', ctx=Load()), attr='array', ctx=Load()), slice=Index(value=Name(id='Any', ctx=Load())), ctx=Load())
                                                                         parse_node_to_type_annotation _typ

                 parse_node_to_type_annotation builtins Tuple(elts=[Name(id='int', ctx=Load()), Subscript(value=Name(id='Literal', ctx=Load()), slice=Index(value=Constant(value=1, kind=None)), ctx=Load())], ctx=Load())
                     parse_node_to_type_annotation builtins Name(id='int', ctx=Load())
                         look_up_name builtins int <function look_up_name at 0x7fcdb80b1dc0>
                         cache hit
                     parse_node_to_type_annotation builtins Subscript(value=Name(id='Literal', ctx=Load()), slice=Index(value=Constant(value=1, kind=None)), ctx=Load())
                         parse_node_to_type_annotation builtins Name(id='Literal', ctx=Load())
                             look_up_name builtins Literal <function look_up_name at 0x7fcdb80b1dc0>
                             cache miss
                                 look_up_name typing_extensions Literal <function look_up_name at 0x7fcdb80b1dc0>
                                 cache hit
    

                             parse_node_to_type_annotation builtins Constant(value=8, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=9, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=10, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=11, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=12, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=13, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=14, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=15, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=16, kind=None)
                             parse_node_to_type_annotation builtins Constant(value=17, kind=None)
                      

                     look_up_name typing Any <function look_up_name at 0x7fcdb80b1dc0>
                     cache hit
             iterate_type_variables_in_type_annotation ConcreteClass(module_name='typing', class_name='Any')
         parse_method ConcreteClass(module_name='builtins', class_name='int') __pow__
             parse_node_to_type_annotation builtins Name(id='int', ctx=Load())
                 look_up_name builtins int <function look_up_name at 0x7fcdb80b1dc0>
                 cache hit
             iterate_type_variables_in_type_annotation ConcreteClass(module_name='builtins', class_name='int')
             parse_node_to_type_annotation builtins Name(id='int', ctx=Load())
                 look_up_name builtins int <function look_up_name at 0x7fcdb80b1dc0>
                 cache hit
             iterate_type_variables_in_type_annotation ConcreteClass(module_name='builtins', class_name='int')
             parse_node_to_type_annotation builtins Name(id='int', ctx=Load())
    

In [59]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [60]:
print(numba_ssa_ir_instructions(ir)[25])

bool44 = global(bool: <class 'bool'>)


In [61]:
handle_instruction(numba_ssa_ir_instructions(ir)[25])

 handle_instruction bool44 = global(bool: <class 'bool'>)
     handle_assign bool44 = global(bool: <class 'bool'>)
         handle_assign_global bool44 = global(bool: <class 'bool'>)
             get_type_handle_for_variable bool44
             set_to_type_annotation <__main__.TypeHandle object at 0x7fcd90e60c70> TypeInferenceConcreteClassObject(concrete_class=ConcreteClass(module_name='builtins', class_name='bool'))
             None -> TypeInferenceConcreteClassObject(concrete_class=ConcreteClass(module_name='builtins', class_name='bool'))


In [62]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [63]:
print(numba_ssa_ir_instructions(ir)[26])

$44pred = call bool44($42compare_op.5, func=bool44, args=(Var($42compare_op.5, 861065169.py:6),), kws=(), vararg=None, varkwarg=None, target=None)


In [64]:
handle_instruction(numba_ssa_ir_instructions(ir)[26])

 handle_instruction $44pred = call bool44($42compare_op.5, func=bool44, args=(Var($42compare_op.5, 861065169.py:6),), kws=(), vararg=None, varkwarg=None, target=None)
     handle_assign $44pred = call bool44($42compare_op.5, func=bool44, args=(Var($42compare_op.5, 861065169.py:6),), kws=(), vararg=None, varkwarg=None, target=None)
         handle_assign_expr $44pred = call bool44($42compare_op.5, func=bool44, args=(Var($42compare_op.5, 861065169.py:6),), kws=(), vararg=None, varkwarg=None, target=None)
             handle_assign_expr_call $44pred = call bool44($42compare_op.5, func=bool44, args=(Var($42compare_op.5, 861065169.py:6),), kws=(), vararg=None, varkwarg=None, target=None)
                 get_type_handle_for_variable $44pred
                 get_type_handle_for_variable bool44
                 get_type_handle_for_variable $42compare_op.5
                 add_present_method <__main__.TypeHandle object at 0x7fcd90e60c70> __call__ TypeInferenceMethod(parameter_type_handle_list=

In [65]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [66]:
print(numba_ssa_ir_instructions(ir)[27])

branch $44pred, 46, 26


In [67]:
print(numba_ssa_ir_instructions(ir)[28])

temp = getitem(value=collection, index=i.2, fn=<built-in function getitem>)


In [68]:
handle_instruction(numba_ssa_ir_instructions(ir)[28])

 handle_instruction temp = getitem(value=collection, index=i.2, fn=<built-in function getitem>)
     handle_assign temp = getitem(value=collection, index=i.2, fn=<built-in function getitem>)
         handle_assign_expr temp = getitem(value=collection, index=i.2, fn=<built-in function getitem>)
             handle_assign_expr_getitem temp = getitem(value=collection, index=i.2, fn=<built-in function getitem>)
                 get_type_handle_for_variable temp
                 get_type_handle_for_variable collection
                 get_type_handle_for_variable i.2
                 add_present_method <__main__.TypeHandle object at 0x7fcd75e0c3a0> __getitem__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd910b0370>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90a9bd60>)
                 concrete_typing_constraints: ConcreteClass(module_name='typing', class_name='Sized')
 look_up_class C

In [69]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [70]:
print(numba_ssa_ir_instructions(ir)[29])

j = i.2


In [71]:
handle_instruction(numba_ssa_ir_instructions(ir)[29])

 handle_instruction j = i.2
     handle_assign j = i.2
         handle_assign_var j = i.2
             handle_variable_assignment j i.2
                 get_type_handle_for_variable i.2


In [72]:
print(numba_ssa_ir_instructions(ir)[30])

jump 58


In [73]:
print(numba_ssa_ir_instructions(ir)[31])

j.2 = phi(incoming_values=[Var(j.1, 861065169.py:11), Var(j, 861065169.py:8)], incoming_blocks=[82, 46])


In [74]:
handle_instruction(numba_ssa_ir_instructions(ir)[31])

 handle_instruction j.2 = phi(incoming_values=[Var(j.1, 861065169.py:11), Var(j, 861065169.py:8)], incoming_blocks=[82, 46])
     handle_assign j.2 = phi(incoming_values=[Var(j.1, 861065169.py:11), Var(j, 861065169.py:8)], incoming_blocks=[82, 46])
         handle_assign_expr j.2 = phi(incoming_values=[Var(j.1, 861065169.py:11), Var(j, 861065169.py:8)], incoming_blocks=[82, 46])
             handle_assign_expr_phi j.2 = phi(incoming_values=[Var(j.1, 861065169.py:11), Var(j, 861065169.py:8)], incoming_blocks=[82, 46])
                 get_type_handle_for_variable j.1
                 get_type_handle_for_variable j
                 associate_with_type_variable [<__main__.TypeHandle object at 0x7fcd912b2160>, <__main__.TypeHandle object at 0x7fcd910b0370>]
                     recursively_iterate_associated_type_handles <__main__.TypeHandle object at 0x7fcd912b2160>
                     recursively_iterate_associated_type_handles <__main__.TypeHandle object at 0x7fcd910b0370>
            

In [75]:
print(numba_ssa_ir_instructions(ir)[32])

$62compare_op.3 = j.2 >= gap


In [76]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [77]:
handle_instruction(numba_ssa_ir_instructions(ir)[32])

 handle_instruction $62compare_op.3 = j.2 >= gap
     handle_assign $62compare_op.3 = j.2 >= gap
         handle_assign_expr $62compare_op.3 = j.2 >= gap
             handle_assign_expr_binop $62compare_op.3 = j.2 >= gap
                 get_type_handle_for_variable $62compare_op.3
                 get_type_handle_for_variable j.2
                 get_type_handle_for_variable gap
                 add_present_method <__main__.TypeHandle object at 0x7fcd912b26d0> __ge__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd912b26d0>, <__main__.TypeHandle object at 0x7fcd90ade490>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd912b27f0>)
                 concrete_typing_constraints: <type_annotation.TypeVariable object at 0x7fcd912b2490>
                     add_present_method <__main__.TypeHandle object at 0x7fcd75dd9400> __ge__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75dd9400>, <__main__.TypeHan

In [78]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [79]:
print(numba_ssa_ir_instructions(ir)[33])

bool64 = global(bool: <class 'bool'>)


In [80]:
handle_instruction(numba_ssa_ir_instructions(ir)[33])

 handle_instruction bool64 = global(bool: <class 'bool'>)
     handle_assign bool64 = global(bool: <class 'bool'>)
         handle_assign_global bool64 = global(bool: <class 'bool'>)
             get_type_handle_for_variable bool64
             set_to_type_annotation <__main__.TypeHandle object at 0x7fcd911acc70> TypeInferenceConcreteClassObject(concrete_class=ConcreteClass(module_name='builtins', class_name='bool'))
             None -> TypeInferenceConcreteClassObject(concrete_class=ConcreteClass(module_name='builtins', class_name='bool'))


In [81]:
print(numba_ssa_ir_instructions(ir)[34])

$64pred = call bool64($62compare_op.3, func=bool64, args=(Var($62compare_op.3, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)


In [82]:
handle_instruction(numba_ssa_ir_instructions(ir)[34])

 handle_instruction $64pred = call bool64($62compare_op.3, func=bool64, args=(Var($62compare_op.3, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)
     handle_assign $64pred = call bool64($62compare_op.3, func=bool64, args=(Var($62compare_op.3, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)
         handle_assign_expr $64pred = call bool64($62compare_op.3, func=bool64, args=(Var($62compare_op.3, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)
             handle_assign_expr_call $64pred = call bool64($62compare_op.3, func=bool64, args=(Var($62compare_op.3, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)
                 get_type_handle_for_variable $64pred
                 get_type_handle_for_variable bool64
                 get_type_handle_for_variable $62compare_op.3
                 add_present_method <__main__.TypeHandle object at 0x7fcd911acc70> __call__ TypeInferenceMethod(parameter_type_handle_list=

In [83]:
print(numba_ssa_ir_instructions(ir)[35])

branch $64pred, 66, 108


In [84]:
print(numba_ssa_ir_instructions(ir)[36])

$72binary_subtract.4 = j.2 - gap


In [85]:
handle_instruction(numba_ssa_ir_instructions(ir)[36])

 handle_instruction $72binary_subtract.4 = j.2 - gap
     handle_assign $72binary_subtract.4 = j.2 - gap
         handle_assign_expr $72binary_subtract.4 = j.2 - gap
             handle_assign_expr_binop $72binary_subtract.4 = j.2 - gap
                 get_type_handle_for_variable $72binary_subtract.4
                 get_type_handle_for_variable j.2
                 get_type_handle_for_variable gap
                 add_present_method <__main__.TypeHandle object at 0x7fcd912b26d0> __sub__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd912b26d0>, <__main__.TypeHandle object at 0x7fcd90ade490>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd911ac460>)
                 concrete_typing_constraints: <type_annotation.TypeVariable object at 0x7fcd912b2490>
                     add_present_method <__main__.TypeHandle object at 0x7fcd75dd9400> __sub__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75dd9

In [86]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [87]:
print(numba_ssa_ir_instructions(ir)[37])

$74binary_subscr.5 = getitem(value=collection, index=$72binary_subtract.4, fn=<built-in function getitem>)


In [88]:
handle_instruction(numba_ssa_ir_instructions(ir)[37])

 handle_instruction $74binary_subscr.5 = getitem(value=collection, index=$72binary_subtract.4, fn=<built-in function getitem>)
     handle_assign $74binary_subscr.5 = getitem(value=collection, index=$72binary_subtract.4, fn=<built-in function getitem>)
         handle_assign_expr $74binary_subscr.5 = getitem(value=collection, index=$72binary_subtract.4, fn=<built-in function getitem>)
             handle_assign_expr_getitem $74binary_subscr.5 = getitem(value=collection, index=$72binary_subtract.4, fn=<built-in function getitem>)
                 get_type_handle_for_variable $74binary_subscr.5
                 get_type_handle_for_variable collection
                 get_type_handle_for_variable $72binary_subtract.4
                 add_present_method <__main__.TypeHandle object at 0x7fcd75e0c3a0> __getitem__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd911ac460>], return_value_type_handle=<__main__.T

In [89]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [90]:
print(numba_ssa_ir_instructions(ir)[38])

$78compare_op.7 = $74binary_subscr.5 > temp


In [91]:
handle_instruction(numba_ssa_ir_instructions(ir)[38])

 handle_instruction $78compare_op.7 = $74binary_subscr.5 > temp
     handle_assign $78compare_op.7 = $74binary_subscr.5 > temp
         handle_assign_expr $78compare_op.7 = $74binary_subscr.5 > temp
             handle_assign_expr_binop $78compare_op.7 = $74binary_subscr.5 > temp
                 get_type_handle_for_variable $78compare_op.7
                 get_type_handle_for_variable $74binary_subscr.5
                 get_type_handle_for_variable temp
                 add_present_method <__main__.TypeHandle object at 0x7fcd911ac2e0> __gt__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd911ac2e0>, <__main__.TypeHandle object at 0x7fcd90a9bd60>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd912d15e0>)
                 concrete_typing_constraints: None


In [92]:
print(numba_ssa_ir_instructions(ir)[39])

bool80 = global(bool: <class 'bool'>)


In [93]:
handle_instruction(numba_ssa_ir_instructions(ir)[39])

 handle_instruction bool80 = global(bool: <class 'bool'>)
     handle_assign bool80 = global(bool: <class 'bool'>)
         handle_assign_global bool80 = global(bool: <class 'bool'>)
             get_type_handle_for_variable bool80
             set_to_type_annotation <__main__.TypeHandle object at 0x7fcd912d1280> TypeInferenceConcreteClassObject(concrete_class=ConcreteClass(module_name='builtins', class_name='bool'))
             None -> TypeInferenceConcreteClassObject(concrete_class=ConcreteClass(module_name='builtins', class_name='bool'))


In [94]:
print(numba_ssa_ir_instructions(ir)[40])

$80pred = call bool80($78compare_op.7, func=bool80, args=(Var($78compare_op.7, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)


In [95]:
handle_instruction(numba_ssa_ir_instructions(ir)[40])

 handle_instruction $80pred = call bool80($78compare_op.7, func=bool80, args=(Var($78compare_op.7, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)
     handle_assign $80pred = call bool80($78compare_op.7, func=bool80, args=(Var($78compare_op.7, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)
         handle_assign_expr $80pred = call bool80($78compare_op.7, func=bool80, args=(Var($78compare_op.7, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)
             handle_assign_expr_call $80pred = call bool80($78compare_op.7, func=bool80, args=(Var($78compare_op.7, 861065169.py:9),), kws=(), vararg=None, varkwarg=None, target=None)
                 get_type_handle_for_variable $80pred
                 get_type_handle_for_variable bool80
                 get_type_handle_for_variable $78compare_op.7
                 add_present_method <__main__.TypeHandle object at 0x7fcd912d1280> __call__ TypeInferenceMethod(parameter_type_handle_list=

In [96]:
print(numba_ssa_ir_instructions(ir)[41])

branch $80pred, 82, 108


In [97]:
print(numba_ssa_ir_instructions(ir)[42])

$88binary_subtract.4 = j.2 - gap


In [98]:
handle_instruction(numba_ssa_ir_instructions(ir)[42])

 handle_instruction $88binary_subtract.4 = j.2 - gap
     handle_assign $88binary_subtract.4 = j.2 - gap
         handle_assign_expr $88binary_subtract.4 = j.2 - gap
             handle_assign_expr_binop $88binary_subtract.4 = j.2 - gap
                 get_type_handle_for_variable $88binary_subtract.4
                 get_type_handle_for_variable j.2
                 get_type_handle_for_variable gap
                 add_present_method <__main__.TypeHandle object at 0x7fcd912b26d0> __sub__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd912b26d0>, <__main__.TypeHandle object at 0x7fcd90ade490>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd91170220>)
                 concrete_typing_constraints: <type_annotation.TypeVariable object at 0x7fcd912b2490>
                     add_present_method <__main__.TypeHandle object at 0x7fcd75dd9400> __sub__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75dd9

In [99]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [100]:
print(numba_ssa_ir_instructions(ir)[43])

$90binary_subscr.5 = getitem(value=collection, index=$88binary_subtract.4, fn=<built-in function getitem>)


In [101]:
handle_instruction(numba_ssa_ir_instructions(ir)[43])

 handle_instruction $90binary_subscr.5 = getitem(value=collection, index=$88binary_subtract.4, fn=<built-in function getitem>)
     handle_assign $90binary_subscr.5 = getitem(value=collection, index=$88binary_subtract.4, fn=<built-in function getitem>)
         handle_assign_expr $90binary_subscr.5 = getitem(value=collection, index=$88binary_subtract.4, fn=<built-in function getitem>)
             handle_assign_expr_getitem $90binary_subscr.5 = getitem(value=collection, index=$88binary_subtract.4, fn=<built-in function getitem>)
                 get_type_handle_for_variable $90binary_subscr.5
                 get_type_handle_for_variable collection
                 get_type_handle_for_variable $88binary_subtract.4
                 add_present_method <__main__.TypeHandle object at 0x7fcd75e0c3a0> __getitem__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd91170220>], return_value_type_handle=<__main__.T

In [102]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [103]:
print(numba_ssa_ir_instructions(ir)[44])

collection[j.2] = $90binary_subscr.5


In [104]:
handle_instruction(numba_ssa_ir_instructions(ir)[44])

 handle_instruction collection[j.2] = $90binary_subscr.5
     handle_setitem collection[j.2] = $90binary_subscr.5
         get_type_handle_for_variable collection
         get_type_handle_for_variable j.2
         get_type_handle_for_variable $90binary_subscr.5
         add_present_method <__main__.TypeHandle object at 0x7fcd75e0c3a0> __setitem__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd912b26d0>, <__main__.TypeHandle object at 0x7fcd91170c10>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd91170d90>)
         concrete_typing_constraints: TypeInferenceClass(present_method_name_to_type_inference_method_dict={'__len__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90b04af0>), '__getitem__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0

In [105]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [106]:
print(numba_ssa_ir_instructions(ir)[45])

$102inplace_subtract.10 = inplace_binop(fn=<built-in function isub>, immutable_fn=<built-in function sub>, lhs=j.2, rhs=gap, static_lhs=Undefined, static_rhs=Undefined)


In [107]:
handle_instruction(numba_ssa_ir_instructions(ir)[45])

 handle_instruction $102inplace_subtract.10 = inplace_binop(fn=<built-in function isub>, immutable_fn=<built-in function sub>, lhs=j.2, rhs=gap, static_lhs=Undefined, static_rhs=Undefined)
     handle_assign $102inplace_subtract.10 = inplace_binop(fn=<built-in function isub>, immutable_fn=<built-in function sub>, lhs=j.2, rhs=gap, static_lhs=Undefined, static_rhs=Undefined)
         handle_assign_expr $102inplace_subtract.10 = inplace_binop(fn=<built-in function isub>, immutable_fn=<built-in function sub>, lhs=j.2, rhs=gap, static_lhs=Undefined, static_rhs=Undefined)
             handle_assign_expr_inplace_binop $102inplace_subtract.10 = inplace_binop(fn=<built-in function isub>, immutable_fn=<built-in function sub>, lhs=j.2, rhs=gap, static_lhs=Undefined, static_rhs=Undefined)
                 get_type_handle_for_variable $102inplace_subtract.10
                 get_type_handle_for_variable j.2
                 get_type_handle_for_variable gap
                 add_present_method <__ma

In [108]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [109]:
print(numba_ssa_ir_instructions(ir)[46])

j.1 = $102inplace_subtract.10


In [110]:
handle_instruction(numba_ssa_ir_instructions(ir)[46])

 handle_instruction j.1 = $102inplace_subtract.10
     handle_assign j.1 = $102inplace_subtract.10
         handle_assign_var j.1 = $102inplace_subtract.10
             handle_variable_assignment j.1 $102inplace_subtract.10
                 get_type_handle_for_variable j.1
                 get_type_handle_for_variable $102inplace_subtract.10
                 associate_with_type_variable [<__main__.TypeHandle object at 0x7fcd912b2160>, <__main__.TypeHandle object at 0x7fcd90adea90>]
                     recursively_iterate_associated_type_handles <__main__.TypeHandle object at 0x7fcd912b2160>
                     recursively_iterate_associated_type_handles <__main__.TypeHandle object at 0x7fcd90adea90>


In [111]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [112]:
print(numba_ssa_ir_instructions(ir)[47])

jump 58


In [113]:
print(numba_ssa_ir_instructions(ir)[48])

collection[j.2] = temp


In [114]:
handle_instruction(numba_ssa_ir_instructions(ir)[48])

 handle_instruction collection[j.2] = temp
     handle_setitem collection[j.2] = temp
         get_type_handle_for_variable collection
         get_type_handle_for_variable j.2
         get_type_handle_for_variable temp
         add_present_method <__main__.TypeHandle object at 0x7fcd75e0c3a0> __setitem__ TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd912b26d0>, <__main__.TypeHandle object at 0x7fcd90a9bd60>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90e7b9d0>)
         concrete_typing_constraints: TypeInferenceClass(present_method_name_to_type_inference_method_dict={'__len__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90b04af0>), '__getitem__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle objec

In [115]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

In [116]:
print(numba_ssa_ir_instructions(ir)[49])

$const118.5 = const(int, 1)


In [117]:
handle_instruction(numba_ssa_ir_instructions(ir)[49])

 handle_instruction $const118.5 = const(int, 1)
     handle_assign $const118.5 = const(int, 1)
         handle_assign_const $const118.5 = const(int, 1)
             get_type_handle_for_variable $const118.5
             set_to_type_annotation <__main__.TypeHandle object at 0x7fcd90e7b610> ConcreteClass(module_name='builtins', class_name='int')
             None -> ConcreteClass(module_name='builtins', class_name='int')


In [118]:
print(numba_ssa_ir_instructions(ir)[50])

$120inplace_add.6 = inplace_binop(fn=<built-in function iadd>, immutable_fn=<built-in function add>, lhs=i.2, rhs=$const118.5, static_lhs=Undefined, static_rhs=Undefined)


In [119]:
handle_instruction(numba_ssa_ir_instructions(ir)[50])

 handle_instruction $120inplace_add.6 = inplace_binop(fn=<built-in function iadd>, immutable_fn=<built-in function add>, lhs=i.2, rhs=$const118.5, static_lhs=Undefined, static_rhs=Undefined)
     handle_assign $120inplace_add.6 = inplace_binop(fn=<built-in function iadd>, immutable_fn=<built-in function add>, lhs=i.2, rhs=$const118.5, static_lhs=Undefined, static_rhs=Undefined)
         handle_assign_expr $120inplace_add.6 = inplace_binop(fn=<built-in function iadd>, immutable_fn=<built-in function add>, lhs=i.2, rhs=$const118.5, static_lhs=Undefined, static_rhs=Undefined)
             handle_assign_expr_inplace_binop $120inplace_add.6 = inplace_binop(fn=<built-in function iadd>, immutable_fn=<built-in function add>, lhs=i.2, rhs=$const118.5, static_lhs=Undefined, static_rhs=Undefined)
                 get_type_handle_for_variable $120inplace_add.6
                 get_type_handle_for_variable i.2
                 get_type_handle_for_variable $const118.5
                 add_present_me

In [120]:
print(numba_ssa_ir_instructions(ir)[51])

i.1 = $120inplace_add.6


In [121]:
handle_instruction(numba_ssa_ir_instructions(ir)[51])

 handle_instruction i.1 = $120inplace_add.6
     handle_assign i.1 = $120inplace_add.6
         handle_assign_var i.1 = $120inplace_add.6
             handle_variable_assignment i.1 $120inplace_add.6
                 get_type_handle_for_variable i.1
                 get_type_handle_for_variable $120inplace_add.6
                 associate_with_type_variable [<__main__.TypeHandle object at 0x7fcd910b0d60>, <__main__.TypeHandle object at 0x7fcd90ad8220>]
                     recursively_iterate_associated_type_handles <__main__.TypeHandle object at 0x7fcd910b0d60>
                     recursively_iterate_associated_type_handles <__main__.TypeHandle object at 0x7fcd90ad8220>


In [122]:
print(numba_ssa_ir_instructions(ir)[52])

jump 34


In [123]:
print(numba_ssa_ir_instructions(ir)[53])

$const128.0 = const(NoneType, None)


In [124]:
handle_instruction(numba_ssa_ir_instructions(ir)[53])

 handle_instruction $const128.0 = const(NoneType, None)
     handle_assign $const128.0 = const(NoneType, None)
         handle_assign_const $const128.0 = const(NoneType, None)
             get_type_handle_for_variable $const128.0
             set_to_type_annotation <__main__.TypeHandle object at 0x7fcd90ad8b80> ConcreteClass(module_name='builtins', class_name='NoneType')
             None -> ConcreteClass(module_name='builtins', class_name='NoneType')


In [125]:
print(numba_ssa_ir_instructions(ir)[54])

$130return_value.1 = cast(value=$const128.0)


In [126]:
handle_instruction(numba_ssa_ir_instructions(ir)[54])

 handle_instruction $130return_value.1 = cast(value=$const128.0)
     handle_assign $130return_value.1 = cast(value=$const128.0)
         handle_assign_expr $130return_value.1 = cast(value=$const128.0)
             handle_assign_expr_cast $130return_value.1 = cast(value=$const128.0)
                 handle_variable_assignment $130return_value.1 $const128.0
                     get_type_handle_for_variable $const128.0


In [127]:
print(numba_ssa_ir_instructions(ir)[55])

return $130return_value.1


In [128]:
variable_name_to_type_handle_dict, type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({'collection': <__main__.TypeHandle at 0x7fcd75e0c3a0>,
  '$const2.0': <__main__.TypeHandle at 0x7fcd75dd9250>,
  '$const4.1': <__main__.TypeHandle at 0x7fcd75dd9310>,
  '$const6.2': <__main__.TypeHandle at 0x7fcd75dd9340>,
  '$const8.3': <__main__.TypeHandle at 0x7fcd75dd9370>,
  '$const10.4': <__main__.TypeHandle at 0x7fcd75dd93d0>,
  '$const12.5': <__main__.TypeHandle at 0x7fcd75dd9400>,
  '$const14.6': <__main__.TypeHandle at 0x7fcd75dd9430>,
  '$const16.7': <__main__.TypeHandle at 0x7fcd75dd9460>,
  'gaps': <__main__.TypeHandle at 0x7fcd75dd96d0>,
  '$24get_iter.10': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$phi26.0': <__main__.TypeHandle at 0x7fcd75dcb250>,
  '$26for_iter.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.2': <__main__.TypeHandle at 0x7fcd90ade490>,
  '$26for_iter.3': <__main__.TypeHandle at 0x7fcd90bfd1c0>,
  '$phi28.1': <__main__.TypeHandle at 0x7fcd90ade490>,
  'gap': <__main__.TypeHandle at 0x7fcd90ade490>,
  'i': <__main__.TypeHandle at 0x7fcd90

## Inferring Types for Variables from Typing Constraints

### Simplify Type Variables

In [129]:
simplify_type_variables()

 simplify_type_variables
 old_type_variable_to_new_type_variable_dict {<type_annotation.TypeVariable object at 0x7fcd910b0250>: <type_annotation.TypeVariable object at 0x7fcd90ef06a0>, <type_annotation.TypeVariable object at 0x7fcd912b2490>: <type_annotation.TypeVariable object at 0x7fcd90ef06a0>, <type_annotation.TypeVariable object at 0x7fcd911ac970>: <type_annotation.TypeVariable object at 0x7fcd90ef06a0>, <type_annotation.TypeVariable object at 0x7fcd911aca30>: <type_annotation.TypeVariable object at 0x7fcd90ef0100>, <type_annotation.TypeVariable object at 0x7fcd911705e0>: <type_annotation.TypeVariable object at 0x7fcd90ef0e50>, <type_annotation.TypeVariable object at 0x7fcd91170f40>: <type_annotation.TypeVariable object at 0x7fcd90ef06a0>, <type_annotation.TypeVariable object at 0x7fcd91170a60>: <type_annotation.TypeVariable object at 0x7fcd90ef0e50>, <type_annotation.TypeVariable object at 0x7fcd91170250>: <type_annotation.TypeVariable object at 0x7fcd90ef06a0>, <type_annotation.

In [130]:
type_handle_to_concrete_typing_constraints_dict, type_variable_to_type_handle_set_dict

({<__main__.TypeHandle at 0x7fcd75e0c3a0>: TypeInferenceClass(present_method_name_to_type_inference_method_dict={'__len__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90b04af0>), '__getitem__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd911702b0>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd911702e0>), '__setitem__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd90e7b2e0>, <__main__.TypeHandle object at 0x7fcd90e7bd00>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90e7bfa0>)}, absent_method_name_to_type_inference_method_dict={}),
  <__main__.TypeHandle at 0x7fcd75dd9250>: ConcreteClass(module_name='builtins', class_name='int'),
  <__main__.TypeHandle at 0x7fcd75

### First Query (Get Classes with Methods)

In [131]:
from settrie import SetTrieMultiMap

from look_up import *
from type_annotation import *


# Initialize CLASS_INHERITANCE_GRAPH by looking up all classes in 'builtins' (and recursively looks up their base classes)
module_name = 'builtins'
import importlib
module = importlib.import_module(module_name)
class_list = [ v for k, v in module.__dict__.items() if isinstance(v, type) and v.__module__ == module_name ]
for c in class_list:
    look_up_class(ConcreteClass(c.__module__, c.__name__))


# Look up all classes in CLASS_INHERITANCE_GRAPH and initialize method_name_set_to_concrete_class_settriemultimap
method_name_set_to_concrete_class_list = []

for concrete_class in CLASS_INHERITANCE_GRAPH.nodes:
    method_name_set = set()
    
    class_definition = look_up_class(concrete_class)
    method_name_set.update(class_definition.method_name_to_method_list_dict)
    
    method_name_set_to_concrete_class_list.append((method_name_set, concrete_class)) 

method_name_set_to_concrete_class_settriemultimap = SetTrieMultiMap(method_name_set_to_concrete_class_list)

def get_classes_with_methods(method_name_iterator):
    method_name_set = set(method_name_iterator)
    viable_method_name_set_to_concrete_class_list = method_name_set_to_concrete_class_settriemultimap.supersets(method_name_set)
    viable_method_name_set_to_concrete_class_list.sort(key=lambda x: len(x[0]))
    return [x[1] for x in viable_method_name_set_to_concrete_class_list]

 look_up_class ConcreteClass(module_name='builtins', class_name='bool')
 cache hit
 look_up_class ConcreteClass(module_name='builtins', class_name='memoryview')
 cache miss
     parse_class ConcreteClass(module_name='builtins', class_name='memoryview') dict_keys(['format', 'itemsize', 'shape', 'strides', 'suboffsets', 'readonly', 'ndim', 'obj', 'c_contiguous', 'f_contiguous', 'contiguous', 'nbytes', '__init__', '__enter__', '__exit__', 'cast', '__getitem__', '__contains__', '__iter__', '__len__', '__setitem__', 'tobytes', 'tolist', 'toreadonly', 'release', 'hex'])
         parse_node_to_type_annotation builtins Subscript(value=Name(id='Sequence', ctx=Load()), slice=Index(value=Name(id='int', ctx=Load())), ctx=Load())
             parse_node_to_type_annotation builtins Name(id='Sequence', ctx=Load())
                 look_up_name builtins Sequence <function look_up_name at 0x7fcdb80b1dc0>
                 cache miss
                     look_up_name typing Sequence <function look_up_nam

                     parse_node_to_type_annotation builtins Name(id='BaseException', ctx=Load())
                         look_up_name builtins BaseException <function look_up_name at 0x7fcdb80b1dc0>
                         cache miss
                 parse_node_to_type_annotation builtins Constant(value=None, kind=None)
             iterate_type_variables_in_type_annotation (Subscription(concrete_class=ConcreteClass(module_name='builtins', class_name='type'), type_annotation_tuple=(ConcreteClass(module_name='builtins', class_name='BaseException'),)), ConcreteClass(module_name='builtins', class_name='NoneType'))
                 iterate_type_variables_in_type_annotation Subscription(concrete_class=ConcreteClass(module_name='builtins', class_name='type'), type_annotation_tuple=(ConcreteClass(module_name='builtins', class_name='BaseException'),))
                     iterate_type_variables_in_type_annotation ConcreteClass(module_name='builtins', class_name='BaseException')
             

                 replace_type_variables_in_type_annotation <type_annotation.TypeVariable object at 0x7fcd91059c70>, {<type_annotation.TypeVariable object at 0x7fcd91059c70>: ConcreteClass(module_name='builtins', class_name='int')}
             replace_type_variables_in_type_annotation ConcreteClass(module_name='builtins', class_name='NoneType'), {<type_annotation.TypeVariable object at 0x7fcd91059c70>: ConcreteClass(module_name='builtins', class_name='int')}
             replace_type_variables_in_type_annotation ConcreteClass(module_name='builtins', class_name='NoneType'), {<type_annotation.TypeVariable object at 0x7fcd91059c70>: ConcreteClass(module_name='builtins', class_name='int')}
             replace_type_variables_in_type_annotation ConcreteClass(module_name='builtins', class_name='int'), {<type_annotation.TypeVariable object at 0x7fcd91059c70>: ConcreteClass(module_name='builtins', class_name='int')}
             replace_type_variables_in_type_annotation Subscription(concrete_c

                 look_up_name typing ByteString <function look_up_name at 0x7fcdb80b1dc0>
                 cache miss
     looking up base class ConcreteClass(module_name='typing', class_name='ByteString')
         look_up_class ConcreteClass(module_name='typing', class_name='ByteString')
         cache miss
             parse_class ConcreteClass(module_name='typing', class_name='ByteString') dict_keys([])
                 parse_node_to_type_annotation typing Subscript(value=Name(id='Sequence', ctx=Load()), slice=Index(value=Name(id='int', ctx=Load())), ctx=Load())
                     parse_node_to_type_annotation typing Name(id='Sequence', ctx=Load())
                         look_up_name typing Sequence <function look_up_name at 0x7fcdb80b1dc0>
                         cache hit
                     parse_node_to_type_annotation typing Name(id='int', ctx=Load())
                         look_up_name typing int <function look_up_name at 0x7fcdb80b1dc0>
                         cache 

     parse_class ConcreteClass(module_name='builtins', class_name='bytes') dict_keys(['__new__', 'capitalize', 'center', 'count', 'decode', 'endswith', 'expandtabs', 'find', 'hex', 'index', 'isalnum', 'isalpha', 'isascii', 'isdigit', 'islower', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill', 'fromhex', 'maketrans', '__len__', '__iter__', '__getitem__', '__add__', '__mul__', '__rmul__', '__mod__', '__contains__', '__eq__', '__ne__', '__lt__', '__le__', '__gt__', '__ge__', '__getnewargs__'])
         parse_node_to_type_annotation builtins Name(id='ByteString', ctx=Load())
             look_up_name builtins ByteString <function look_up_name at 0x7fcdb80b1dc0>
             cache hit
     looking up base class ConcreteClass(module_name='typing', class_name='ByteString')
         look_up_cl

...

     parse_class ConcreteClass(module_name='builtins', class_name='ImportWarning') dict_keys([])
         parse_node_to_type_annotation builtins Name(id='Warning', ctx=Load())
             look_up_name builtins Warning <function look_up_name at 0x7fcdb80b1dc0>
             cache hit
     looking up base class ConcreteClass(module_name='builtins', class_name='Warning')
         look_up_class ConcreteClass(module_name='builtins', class_name='Warning')
         cache hit
     instantiating type variables in base class ConcreteClass(module_name='builtins', class_name='Warning') with []
         instantiate_type_variables_in_class_definition ClassDefinition(type_variable_list=[], method_name_to_method_list_dict={'__init__': [FunctionDefinition(type_variable_list=[], parameter_type_annotation_list=[ConcreteClass(module_name='builtins', class_name='Warning')], vararg_type_annotation=ConcreteClass(module_name='builtins', class_name='object'), kwonlyargs_name_to_type_annotation_dict={}, kwarg_t

     parse_class ConcreteClass(module_name='builtins', class_name='ConnectionError') dict_keys([])
         parse_node_to_type_annotation builtins Name(id='OSError', ctx=Load())
             look_up_name builtins OSError <function look_up_name at 0x7fcdb80b1dc0>
             cache miss
     looking up base class ConcreteClass(module_name='builtins', class_name='OSError')
         look_up_class ConcreteClass(module_name='builtins', class_name='OSError')
         cache hit
     instantiating type variables in base class ConcreteClass(module_name='builtins', class_name='OSError') with []
         instantiate_type_variables_in_class_definition ClassDefinition(type_variable_list=[], method_name_to_method_list_dict={'__init__': [FunctionDefinition(type_variable_list=[], parameter_type_annotation_list=[ConcreteClass(module_name='builtins', class_name='OSError')], vararg_type_annotation=ConcreteClass(module_name='builtins', class_name='object'), kwonlyargs_name_to_type_annotation_dict={}, kwar

     parse_class ConcreteClass(module_name='builtins', class_name='ChildProcessError') dict_keys([])
         parse_node_to_type_annotation builtins Name(id='OSError', ctx=Load())
             look_up_name builtins OSError <function look_up_name at 0x7fcdb80b1dc0>
             cache hit
     looking up base class ConcreteClass(module_name='builtins', class_name='OSError')
         look_up_class ConcreteClass(module_name='builtins', class_name='OSError')
         cache hit
     instantiating type variables in base class ConcreteClass(module_name='builtins', class_name='OSError') with []
         instantiate_type_variables_in_class_definition ClassDefinition(type_variable_list=[], method_name_to_method_list_dict={'__init__': [FunctionDefinition(type_variable_list=[], parameter_type_annotation_list=[ConcreteClass(module_name='builtins', class_name='OSError')], vararg_type_annotation=ConcreteClass(module_name='builtins', class_name='object'), kwonlyargs_name_to_type_annotation_dict={}, kwa

     parse_class ConcreteClass(module_name='builtins', class_name='FileExistsError') dict_keys([])
         parse_node_to_type_annotation builtins Name(id='OSError', ctx=Load())
             look_up_name builtins OSError <function look_up_name at 0x7fcdb80b1dc0>
             cache hit
     looking up base class ConcreteClass(module_name='builtins', class_name='OSError')
         look_up_class ConcreteClass(module_name='builtins', class_name='OSError')
         cache hit
     instantiating type variables in base class ConcreteClass(module_name='builtins', class_name='OSError') with []
         instantiate_type_variables_in_class_definition ClassDefinition(type_variable_list=[], method_name_to_method_list_dict={'__init__': [FunctionDefinition(type_variable_list=[], parameter_type_annotation_list=[ConcreteClass(module_name='builtins', class_name='OSError')], vararg_type_annotation=ConcreteClass(module_name='builtins', class_name='object'), kwonlyargs_name_to_type_annotation_dict={}, kwarg

     parse_class ConcreteClass(module_name='builtins', class_name='PermissionError') dict_keys([])
         parse_node_to_type_annotation builtins Name(id='OSError', ctx=Load())
             look_up_name builtins OSError <function look_up_name at 0x7fcdb80b1dc0>
             cache hit
     looking up base class ConcreteClass(module_name='builtins', class_name='OSError')
         look_up_class ConcreteClass(module_name='builtins', class_name='OSError')
         cache hit
     instantiating type variables in base class ConcreteClass(module_name='builtins', class_name='OSError') with []
         instantiate_type_variables_in_class_definition ClassDefinition(type_variable_list=[], method_name_to_method_list_dict={'__init__': [FunctionDefinition(type_variable_list=[], parameter_type_annotation_list=[ConcreteClass(module_name='builtins', class_name='OSError')], vararg_type_annotation=ConcreteClass(module_name='builtins', class_name='object'), kwonlyargs_name_to_type_annotation_dict={}, kwarg

In [132]:
type_variable_to_possible_class_list_dict = dict()
type_handle_to_type_variable_dict = dict()

for type_variable, type_handle_set in type_variable_to_type_handle_set_dict.items():
    present_method_set = set()
    
    for type_handle in type_handle_set:
        type_handle_to_type_variable_dict[type_handle] = type_variable
        
        implementation = type_handle_to_concrete_typing_constraints_dict[type_handle]
        if isinstance(implementation, TypeInferenceClass):
            present_method_set.update(implementation.present_method_name_to_type_inference_method_dict)
    
    type_variable_to_possible_class_list_dict[type_variable] = get_classes_with_methods(present_method_set)

In [133]:
type_variable_to_possible_class_list_dict

{<type_annotation.TypeVariable at 0x7fcd90ef0100>: [ConcreteClass(module_name='typing', class_name='AbstractSet'),
  ConcreteClass(module_name='builtins', class_name='tuple'),
  ConcreteClass(module_name='builtins', class_name='frozenset'),
  ConcreteClass(module_name='typing', class_name='MutableSet'),
  ConcreteClass(module_name='builtins', class_name='list'),
  ConcreteClass(module_name='builtins', class_name='set'),
  ConcreteClass(module_name='builtins', class_name='bool'),
  ConcreteClass(module_name='builtins', class_name='int'),
  ConcreteClass(module_name='builtins', class_name='bytes'),
  ConcreteClass(module_name='builtins', class_name='str'),
  ConcreteClass(module_name='builtins', class_name='bytearray')]}

In [134]:
remaining_type_handle_pointing_to_type_inference_class_to_possible_class_list_dict = dict()
for type_handle, implementation in type_handle_to_concrete_typing_constraints_dict.items():
    if isinstance(implementation, TypeInferenceClass) and type_handle not in type_handle_to_type_variable_dict:
        remaining_type_handle_pointing_to_type_inference_class_to_possible_class_list_dict[type_handle] = get_classes_with_methods(
            implementation.present_method_name_to_type_inference_method_dict
        )

In [135]:
remaining_type_handle_pointing_to_type_inference_class_to_possible_class_list_dict

{<__main__.TypeHandle at 0x7fcd75e0c3a0>: [ConcreteClass(module_name='typing', class_name='MutableMapping'),
  ConcreteClass(module_name='typing', class_name='MutableSequence'),
  ConcreteClass(module_name='builtins', class_name='dict'),
  ConcreteClass(module_name='builtins', class_name='list'),
  ConcreteClass(module_name='builtins', class_name='memoryview'),
  ConcreteClass(module_name='builtins', class_name='bytearray')],
 <__main__.TypeHandle at 0x7fcd910b0d60>: [ConcreteClass(module_name='builtins', class_name='bool'),
  ConcreteClass(module_name='builtins', class_name='int')],
 <__main__.TypeHandle at 0x7fcd912b2160>: [ConcreteClass(module_name='typing', class_name='AbstractSet'),
  ConcreteClass(module_name='builtins', class_name='frozenset'),
  ConcreteClass(module_name='typing', class_name='MutableSet'),
  ConcreteClass(module_name='builtins', class_name='set'),
  ConcreteClass(module_name='builtins', class_name='bool'),
  ConcreteClass(module_name='builtins', class_name='int

### Second Query

#### Query Generation

In [136]:
type_variable_to_second_query_dict = dict()

for type_variable, type_handle_set in type_variable_to_type_handle_set_dict.items():
    type_inference_class_list = list()
        
    for type_handle in type_handle_set:
        implementation = type_handle_to_concrete_typing_constraints_dict[type_handle]
        if isinstance(implementation, TypeInferenceClass):
            type_inference_class_list.append(implementation)
    
    if type_inference_class_list:
        second_query = create_second_query(type_inference_class_list)
    else:
        second_query = dict()
    
    type_variable_to_second_query_dict[type_variable] = second_query

 create_second_query [TypeInferenceClass(present_method_name_to_type_inference_method_dict={'__gt__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd911ac2e0>, <__main__.TypeHandle object at 0x7fcd90a9bd60>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd912d15e0>)}, absent_method_name_to_type_inference_method_dict={})]
     create_method_query TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd911ac2e0>, <__main__.TypeHandle object at 0x7fcd90a9bd60>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd912d15e0>)
         create_parameter_or_return_value_constraint <__main__.TypeHandle object at 0x7fcd90a9bd60>
         create_parameter_or_return_value_constraint <__main__.TypeHandle object at 0x7fcd912d15e0>


In [137]:
type_variable_to_second_query_dict

{<type_annotation.TypeVariable at 0x7fcd90ef0100>: {'__gt__': {'number_of_parameters': 2,
   'parameter_indices_to_parameter_constraints': {1: None},
   'return_value_constraint': ConcreteClass(module_name='builtins', class_name='object')}}}

In [138]:
remaining_type_handle_pointing_to_type_inference_class_to_second_query_dict = dict()

for remaining_type_handle_pointing_to_type_inference_class in remaining_type_handle_pointing_to_type_inference_class_to_possible_class_list_dict:
    type_inference_class = type_handle_to_concrete_typing_constraints_dict[remaining_type_handle_pointing_to_type_inference_class]
    second_query = create_second_query([type_inference_class])
    remaining_type_handle_pointing_to_type_inference_class_to_second_query_dict[
        remaining_type_handle_pointing_to_type_inference_class
    ] = second_query

 create_second_query [TypeInferenceClass(present_method_name_to_type_inference_method_dict={'__len__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90b04af0>), '__getitem__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd911702b0>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd911702e0>), '__setitem__': TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>, <__main__.TypeHandle object at 0x7fcd90e7b2e0>, <__main__.TypeHandle object at 0x7fcd90e7bd00>], return_value_type_handle=<__main__.TypeHandle object at 0x7fcd90e7bfa0>)}, absent_method_name_to_type_inference_method_dict={})]
     create_method_query TypeInferenceMethod(parameter_type_handle_list=[<__main__.TypeHandle object at 0x7fcd75e0c3a0>], return_value_type_handle=<__main_

In [139]:
remaining_type_handle_pointing_to_type_inference_class_to_second_query_dict

{<__main__.TypeHandle at 0x7fcd75e0c3a0>: {'__len__': {'number_of_parameters': 1,
   'parameter_indices_to_parameter_constraints': {},
   'return_value_constraint': ConcreteClass(module_name='builtins', class_name='int')},
  '__getitem__': {'number_of_parameters': 2,
   'parameter_indices_to_parameter_constraints': {1: ConcreteClass(module_name='builtins', class_name='int')},
   'return_value_constraint': <type_annotation.TypeVariable at 0x7fcd90ef0100>},
  '__setitem__': {'number_of_parameters': 3,
   'parameter_indices_to_parameter_constraints': {1: ConcreteClass(module_name='builtins', class_name='int'),
    2: <type_annotation.TypeVariable at 0x7fcd90ef0100>},
   'return_value_constraint': ConcreteClass(module_name='builtins', class_name='NoneType')}},
 <__main__.TypeHandle at 0x7fcd910b0d60>: {'__lt__': {'number_of_parameters': 2,
   'parameter_indices_to_parameter_constraints': {1: ConcreteClass(module_name='builtins', class_name='int')},
   'return_value_constraint': ConcreteCla

#### Query Results

In [140]:
remaining_type_handle_pointing_to_type_inference_class_to_second_query_result_dict = {
    remaining_type_handle_pointing_to_type_inference_class: execute_second_query(
        second_query,
        remaining_type_handle_pointing_to_type_inference_class_to_possible_class_list_dict[
            remaining_type_handle_pointing_to_type_inference_class
        ]
    )
    for remaining_type_handle_pointing_to_type_inference_class, second_query
    in remaining_type_handle_pointing_to_type_inference_class_to_second_query_dict.items()
}

 execute_second_query {'__len__': {'number_of_parameters': 1, 'parameter_indices_to_parameter_constraints': {}, 'return_value_constraint': ConcreteClass(module_name='builtins', class_name='int')}, '__getitem__': {'number_of_parameters': 2, 'parameter_indices_to_parameter_constraints': {1: ConcreteClass(module_name='builtins', class_name='int')}, 'return_value_constraint': <type_annotation.TypeVariable object at 0x7fcd90ef0100>}, '__setitem__': {'number_of_parameters': 3, 'parameter_indices_to_parameter_constraints': {1: ConcreteClass(module_name='builtins', class_name='int'), 2: <type_annotation.TypeVariable object at 0x7fcd90ef0100>}, 'return_value_constraint': ConcreteClass(module_name='builtins', class_name='NoneType')}} [ConcreteClass(module_name='typing', class_name='MutableMapping'), ConcreteClass(module_name='typing', class_name='MutableSequence'), ConcreteClass(module_name='builtins', class_name='dict'), ConcreteClass(module_name='builtins', class_name='list'), ConcreteClass(mo

In [141]:
remaining_type_handle_pointing_to_type_inference_class_to_second_query_result_dict

{<__main__.TypeHandle at 0x7fcd75e0c3a0>: {ConcreteClass(module_name='typing', class_name='MutableMapping'): {'__len__': {0: {'parameter_type_annotations': {},
     'return_value_type_annotation': {'type_annotation': ConcreteClass(module_name='builtins', class_name='int'),
      'relation': 'supertype'}}},
   '__getitem__': {0: {'parameter_type_annotations': {1: {'type_annotation': <type_annotation.TypeVariable at 0x7fcd90fc2e20>,
       'relation': 'supertype'}},
     'return_value_type_annotation': {'type_annotation': <type_annotation.TypeVariable at 0x7fcd90f5ab50>,
      'relation': 'supertype'}}},
   '__setitem__': {0: {'parameter_type_annotations': {1: {'type_annotation': <type_annotation.TypeVariable at 0x7fcd90fc2e20>,
       'relation': 'supertype'},
      2: {'type_annotation': <type_annotation.TypeVariable at 0x7fcd90f5ab50>,
       'relation': 'supertype'}},
     'return_value_type_annotation': {'type_annotation': ConcreteClass(module_name='builtins', class_name='NoneType')

In [142]:
type_variable_to_second_query_result_dict = {
    type_variable: execute_second_query(
        second_query,
        type_variable_to_possible_class_list_dict[
            type_variable
        ]
    )
    for type_variable, second_query
    in type_variable_to_second_query_dict.items()
}

 execute_second_query {'__gt__': {'number_of_parameters': 2, 'parameter_indices_to_parameter_constraints': {1: None}, 'return_value_constraint': ConcreteClass(module_name='builtins', class_name='object')}} [ConcreteClass(module_name='typing', class_name='AbstractSet'), ConcreteClass(module_name='builtins', class_name='tuple'), ConcreteClass(module_name='builtins', class_name='frozenset'), ConcreteClass(module_name='typing', class_name='MutableSet'), ConcreteClass(module_name='builtins', class_name='list'), ConcreteClass(module_name='builtins', class_name='set'), ConcreteClass(module_name='builtins', class_name='bool'), ConcreteClass(module_name='builtins', class_name='int'), ConcreteClass(module_name='builtins', class_name='bytes'), ConcreteClass(module_name='builtins', class_name='str'), ConcreteClass(module_name='builtins', class_name='bytearray')]
     create_satisfying_concrete_class {'__gt__': {'number_of_parameters': 2, 'parameter_indices_to_parameter_constraints': {1: None}, 're

In [143]:
type_variable_to_second_query_result_dict

{<type_annotation.TypeVariable at 0x7fcd90ef0100>: {ConcreteClass(module_name='typing', class_name='AbstractSet'): {'__gt__': {0: {'parameter_type_annotations': {1: {'type_annotation': Subscription(concrete_class=ConcreteClass(module_name='typing', class_name='AbstractSet'), type_annotation_tuple=(ConcreteClass(module_name='typing', class_name='Any'),)),
       'relation': 'subtype'}},
     'return_value_type_annotation': {'type_annotation': ConcreteClass(module_name='builtins', class_name='bool'),
      'relation': 'subtype'}}},
   'type_variable_to_type_annotation_dict': {<type_annotation.TypeVariable at 0x7fcd90e7b670>: <type_annotation.TypeVariable at 0x7fcd90e7b670>}},
  ConcreteClass(module_name='builtins', class_name='tuple'): {'__gt__': {0: {'parameter_type_annotations': {1: {'type_annotation': Subscription(concrete_class=ConcreteClass(module_name='builtins', class_name='tuple'), type_annotation_tuple=(<type_annotation.TypeVariable object at 0x7fcd90b0a580>, ConcreteClass(modul

## Results

What are the viable types of the parameter `collection`?

```python
def shell_sort(collection):
    # Marcin Ciura's gap sequence
    gaps = [701, 301, 132, 57, 23, 10, 4, 1]
    for gap in gaps:
        i = gap
        while i < len(collection):
            temp = collection[i]
            j = i
            while j >= gap and collection[j - gap] > temp:
                collection[j] = collection[j - gap]
                j -= gap
            collection[j] = temp
            i += 1
```

In [144]:
collection_type_handle = variable_name_to_type_handle_dict['collection']

Does `collection_type_handle` point to a `TypeHandle`?

In [145]:
collection_type_handle in type_handle_to_type_variable_dict

False

In [146]:
for concrete_class, dict_ in remaining_type_handle_pointing_to_type_inference_class_to_second_query_result_dict[collection_type_handle].items():
    print(concrete_class, dict_['type_variable_to_type_annotation_dict'], '\n')

ConcreteClass(module_name='typing', class_name='MutableMapping') {<type_annotation.TypeVariable object at 0x7fcd90fc2e20>: ConcreteClass(module_name='builtins', class_name='int'), <type_annotation.TypeVariable object at 0x7fcd90f5ab50>: <type_annotation.TypeVariable object at 0x7fcd90ef0100>} 

ConcreteClass(module_name='typing', class_name='MutableSequence') {<type_annotation.TypeVariable object at 0x7fcd91059c70>: <type_annotation.TypeVariable object at 0x7fcd90ef0100>} 

ConcreteClass(module_name='builtins', class_name='dict') {<type_annotation.TypeVariable object at 0x7fcd910605b0>: ConcreteClass(module_name='builtins', class_name='int'), <type_annotation.TypeVariable object at 0x7fcd911f1250>: <type_annotation.TypeVariable object at 0x7fcd90ef0100>} 

ConcreteClass(module_name='builtins', class_name='list') {<type_annotation.TypeVariable object at 0x7fcd9136bb20>: <type_annotation.TypeVariable object at 0x7fcd90ef0100>} 

ConcreteClass(module_name='builtins', class_name='memoryvie

Why? Because of the `second_query` generated from the constraints:

In [147]:
remaining_type_handle_pointing_to_type_inference_class_to_second_query_dict[collection_type_handle]

{'__len__': {'number_of_parameters': 1,
  'parameter_indices_to_parameter_constraints': {},
  'return_value_constraint': ConcreteClass(module_name='builtins', class_name='int')},
 '__getitem__': {'number_of_parameters': 2,
  'parameter_indices_to_parameter_constraints': {1: ConcreteClass(module_name='builtins', class_name='int')},
  'return_value_constraint': <type_annotation.TypeVariable at 0x7fcd90ef0100>},
 '__setitem__': {'number_of_parameters': 3,
  'parameter_indices_to_parameter_constraints': {1: ConcreteClass(module_name='builtins', class_name='int'),
   2: <type_annotation.TypeVariable at 0x7fcd90ef0100>},
  'return_value_constraint': ConcreteClass(module_name='builtins', class_name='NoneType')}}

What about the elements within `collection` (a.k.a. `temp`)?

In [148]:
temp_type_handle = variable_name_to_type_handle_dict['temp']

Does `temp_type_handle` point to a `TypeHandle`?

In [149]:
temp_type_handle in type_handle_to_type_variable_dict

True

In [150]:
temp_type_variable = type_handle_to_type_variable_dict[temp_type_handle]

In [151]:
for concrete_class, dict_ in type_variable_to_second_query_result_dict[temp_type_variable].items():
    print(concrete_class, dict_['type_variable_to_type_annotation_dict'], '\n')

ConcreteClass(module_name='typing', class_name='AbstractSet') {<type_annotation.TypeVariable object at 0x7fcd90e7b670>: <type_annotation.TypeVariable object at 0x7fcd90e7b670>} 

ConcreteClass(module_name='builtins', class_name='tuple') {<type_annotation.TypeVariable object at 0x7fcd90b0a580>: <type_annotation.TypeVariable object at 0x7fcd90b0a580>} 

ConcreteClass(module_name='builtins', class_name='frozenset') {<type_annotation.TypeVariable object at 0x7fcd90b0a580>: <type_annotation.TypeVariable object at 0x7fcd90b0a580>} 

ConcreteClass(module_name='typing', class_name='MutableSet') {<type_annotation.TypeVariable object at 0x7fcd91059c70>: <type_annotation.TypeVariable object at 0x7fcd91059c70>} 

ConcreteClass(module_name='builtins', class_name='list') {<type_annotation.TypeVariable object at 0x7fcd9136bb20>: <type_annotation.TypeVariable object at 0x7fcd9136bb20>} 

ConcreteClass(module_name='builtins', class_name='set') {<type_annotation.TypeVariable object at 0x7fcd9136bb20>: <

Why? Because of the `second_query` generated from the constraints:

In [152]:
type_variable_to_second_query_dict[temp_type_variable]

{'__gt__': {'number_of_parameters': 2,
  'parameter_indices_to_parameter_constraints': {1: None},
  'return_value_constraint': ConcreteClass(module_name='builtins', class_name='object')}}